In [11]:
from pathlib import Path

import nltk
import pandas as pd
from swifter import set_defaults
from sentence_transformers import SentenceTransformer
import numpy as np
from scipy.spatial import distance
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sentence_transformers import util

In [12]:
DATA_PATH = Path("data")
YEAR = 2016
MONTH = 2
DAY_START = 1
DAY_END = 28  # end day is inclusive

In [ ]:
set_defaults(
    npartitions=64,
    dask_threshold=1,
    scheduler="processes",
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=True,
    force_parallel=True,
)
nltk.download("punkt")

# Wstępna analiza

Dane zostały wstępnie przetworzone: pobrano zbiorczy plik csv dla wszystkich lat (2016-2020) a następnie posortowano go po dacie.
Artykuły z kolejnych lat zapisano do odpowiednich plików (np. `data/raw/year_2016.csv`).

Analiza ta obejmuje następujące kroki:
1. Wczytanie danych
2. Usunięcie artykułów z pustym tytułem lub treścią
3. Wybranie artykułów z wybranego miesiąca i dnia (zakres dat jest w zmiennej DAY_START i DAY_END)
4. Usunięcie duplikatów tytułów
5. Podsumowanie liczby artykułów

In [ ]:
raw_df = pd.read_csv(DATA_PATH / f"/by_year/year_{YEAR}.csv")

In [ ]:
raw_df.describe()

In [ ]:
raw_df.head(100)

In [ ]:
# count null in each column
raw_df.isnull().sum()

In [ ]:
# drop titles with null values
raw_df = raw_df.dropna(subset=["title"])
raw_df = raw_df.dropna(subset=["article"])

In [ ]:
df = raw_df[raw_df["month"] == MONTH]
df = df[df["day"] <= DAY_END]
df = df[df["day"] >= DAY_START]

In [ ]:
df["publication"].value_counts()

In [ ]:
print(f"Całkowita liczba artykułów: {len(df)}")
df = df.drop_duplicates(subset=["title"])
df = df.drop_duplicates(subset=["article"])
print(f"Całkowita liczba artykułów po usunięciu duplikatów tytułów: {len(df)}")

# Utworzenie embeddingów

Proces ten obejmuje:
1. Pobranie modelu SentenceTransformer na podstawie all-mpnet-base-v2 (https://www.sbert.net/docs/pretrained_models.html)
2. Utworzenie embeddingów dla każdego zdania w artykule i zapisanie ich do kolumny 'sentence_embeddings'
3. Utworzenie embeddingów dla całego artykułu poprzez uśrednienie embeddingów zdań i zapisanie ich do kolumny 'embedding'
4. Odfiltrowanie artykułów, które nie mają embeddingów (zwykle są to artykuły, które nie zawierają żadnych zdań)
5. Zapisanie dataframe do pliku parquet, tak aby można było go łatwo wczytać w przyszłości

In [14]:
model = SentenceTransformer("all-mpnet-base-v2")

In [15]:
def get_article_embedding(row):
    article = row["article"]
    # Split the article into sentences
    sentences = nltk.sent_tokenize(article)

    # Generate embeddings for each sentence
    sentence_embeddings = model.encode(sentences)

    # Average the sentence embeddings to get a single vector
    article_embedding = np.mean(sentence_embeddings, axis=0)
    row["embedding"] = article_embedding
    row["sentence_embeddings"] = sentence_embeddings.flatten()

    return row

In [ ]:
# calculate embeddings for each article using df.apply
df = df.swifter.apply(get_article_embedding, axis=1)

In [ ]:
df.head(10)

In [ ]:
# make sure every embedding is 768
df = df[df["embedding"].apply(lambda x: x.shape) == (768,)]

In [ ]:
df.to_parquet(DATA_PATH / f"embedded/{YEAR}_embd.parquet")

# Klastrowanie na podstawie embeddingów (AgglomerativeClustering)

Proces ten obejmuje:
1. Wczytanie pliku parquet z embeddingami
2. Odfiltrowanie artykułów z wybranego miesiąca i dnia (zakres dat jest w zmiennej DAY_START i DAY_END)
3. Utworzenie klastrów za pomocą AgglomerativeClustering (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)
4. Odfiltrowanie klastrów, które zawierają tylko jeden artykuł
5. Odfiltrowanie klastrów, w których artykuły pochodzą z tego samego źródła
6. Wyświetlenie tytułów artykułów w wybranych klastrach

In [17]:
# read model once again
model = SentenceTransformer("all-mpnet-base-v2")

In [18]:
MONTH = 2
DAY_START = 5
DAY_END = 5  # end day is inclusive
YEAR = 2016

In [19]:
df = pd.read_parquet(DATA_PATH / f"embedded/{YEAR}_embd.parquet")

In [20]:
# filter date again
df = df[df["month"] == MONTH]
df = df[df["day"] <= DAY_END]
df = df[df["day"] >= DAY_START]

In [21]:
cp_raw = np.vstack(df["embedding"].values)
corpus_embeddings = cp_raw / np.linalg.norm(cp_raw, axis=1, keepdims=True)
clustering_model = AgglomerativeClustering(
    n_clusters=None, distance_threshold=0.2, metric="cosine", linkage="average"
)
clustering_model.fit(corpus_embeddings)
df["cluster"] = clustering_model.labels_

In [22]:
cluster_publisher_counts = df.groupby("cluster")["publication"].nunique()
single_publisher_clusters = cluster_publisher_counts[cluster_publisher_counts == 1].index
filtered_df = df[~df["cluster"].isin(single_publisher_clusters)]

In [23]:
# ograniczenie zestawu danych (w korpusie pojawily sie artykuly tego samego autora do tego samego czasopisma o rozniace sie 1 znakiem w tytule np)
filtered_df['combined']= filtered_df["author"]+filtered_df["title"].str[:15]
filtered_df = filtered_df.drop_duplicates(subset='combined', keep='first')
filtered_df = filtered_df.drop(columns='combined')

C:\Users\User\AppData\Local\Temp\ipykernel_1700\2185096312.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['combined']= filtered_df["author"]+filtered_df["title"].str[:15]


# Weryfikacja jakości dokonanego podziału na klastry

Do weryfikacji podziału na klastry użyto metryki silhouette score (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).

Metryka ta jest obliczana dla każdego punktu na podstawie średniej odległości między próbkami wewnątrz klastra i najbliższego klastra z nim sąsiadującego.
Wartość metryki mieści się w przedziale [-1, 1], gdzie wartości bliskie 1 oznaczają, że punkt jest dobrze dopasowany do klastra, a wartości bliskie -1 oznaczają, że punkt jest źle dopasowany do klastra. Wartości bliskie 0 oznaczają, że punkt jest blisko granicy klastra. Jest to rodzaj metryki wewnętrznej, która może być używana do oceny jakości podziału na klastry bez posiadania dodatkowej wiedzy o zbiorach czy etykiet klastrów.

W tym kroku:
1. Obliczono sumaryczny silhouette score dla wszystkich klastrów.
1. Obliczono silhouette score dla każdego artykułu z osobna
2. Obliczono silhouette score dla każdego klastra z osobna

In [24]:
len(filtered_df)

359

In [25]:
counted_df = filtered_df["cluster"].value_counts()
counted_df.head()
#print(counted_df[21])
#print("Liczba klastrów o wielkości 21 wynosi {}".format(counted_df[counted_df == 132].count()))

cluster
12     20
15     10
9      10
328     8
53      8
Name: count, dtype: int64

In [26]:
score1 = silhouette_score(filtered_df["embedding"].tolist(),filtered_df["cluster"].tolist(),metric="cosine",)
print("Sumaryczny wynik dokładności podziału na klastry dla wszystkich wynosi {:.3f}".format(score1))

Sumaryczny wynik dokładności podziału na klastry dla wszystkich wynosi 0.509


Uzyskany wynik sugeruje, że podział na klastry jest raczej dobry, biorąc pod uwagę rozpatrywane zbiory danych.

In [27]:
# compute the silhouette coefficients for each sample,
sample_silhouette_values = silhouette_samples(filtered_df["embedding"].tolist(),filtered_df["cluster"].tolist(),metric="cosine")
filtered_df.loc[:, "silhouette_values"] = sample_silhouette_values

In [28]:
# Wyświetlenie silhouette samples
for i, silhouette_value in enumerate(sample_silhouette_values):
    print(f"Silhouette dla próbki {i+1}: {silhouette_value}")

Silhouette dla próbki 1: 0.7110530138015747
Silhouette dla próbki 2: 0.7341523170471191
Silhouette dla próbki 3: 0.7950366735458374
Silhouette dla próbki 4: 0.20621824264526367
Silhouette dla próbki 5: 0.4333049952983856
Silhouette dla próbki 6: 0.6667561531066895
Silhouette dla próbki 7: 0.7558681964874268
Silhouette dla próbki 8: 0.5175563097000122
Silhouette dla próbki 9: 0.704115629196167
Silhouette dla próbki 10: 0.7456156611442566
Silhouette dla próbki 11: 0.8234489560127258
Silhouette dla próbki 12: 0.7087992429733276
Silhouette dla próbki 13: 0.9721491932868958
Silhouette dla próbki 14: 0.7580787539482117
Silhouette dla próbki 15: 0.7275242209434509
Silhouette dla próbki 16: 0.7400155067443848
Silhouette dla próbki 17: 0.7437456250190735
Silhouette dla próbki 18: 0.7646558284759521
Silhouette dla próbki 19: 0.42971307039260864
Silhouette dla próbki 20: 0.779112696647644
Silhouette dla próbki 21: 0.6154232025146484
Silhouette dla próbki 22: 0.35774052143096924
Silhouette dla pró

In [29]:
# Iteruj po unikalnych etykietach klastrów
for cluster_id in filtered_df["cluster"].unique():
    # Utwórz kopię ramki danych dla danego klastra
    cluster_subset = filtered_df[filtered_df["cluster"] == cluster_id].copy()

    num_elements = cluster_subset.shape[0]

    # Oblicz średnią wartość silhouette dla danego klastra
    cluster_mean_silhouette = cluster_subset["silhouette_values"].mean()

    # Dodaj średnią wartość silhouette dla danego klastra do wszystkich wierszy tego klastra
    filtered_df.loc[filtered_df["cluster"] == cluster_id, "cluster_mean_silhouette"] = cluster_mean_silhouette
    filtered_df.loc[filtered_df["cluster"] == cluster_id, "cluster_size"] = num_elements
filtered_df

,date,year,month,day,author,title,article,url,section,publication,embedding,sentence_embeddings,cluster,silhouette_values,cluster_mean_silhouette,cluster_size
56284,2016-02-05,2016,2.0,5,Kelly Diamond,Ben Stiller unveils world's largest selfie sti...,The world just welcomed its newest record: The...,https://mashable.com/2016/02/05/ben-stiller-se...,None,Mashable,"[0.008487079, -0.030727869, 0.0062861373, -0.0...","[0.058348633, -0.062717475, 0.019179648, 0.016...",132,0.711053,0.712094,2.0
56291,2016-02-05,2016,2.0,5,None,Famous Japan volcano erupts spectacularly with...,A well-known volcano erupted Friday evening i...,https://mashable.com/2016/02/05/japan-volcano-...,None,Mashable,"[0.001972216, -0.0141342925, 0.0026705277, 0.0...","[0.03175077, 0.0061671427, 0.020649543, 0.0318...",88,0.734152,0.749286,5.0
56294,2016-02-05,2016,2.0,5,David Shepardson,U.S. traffic deaths increase sharply in 2015,"WASHINGTON, Feb 5 (Reuters) - Traffic deaths i...",http://www.reuters.com/article/usa-trafficdeat...,Intel,Reuters,"[-0.02990842, 0.049623482, -0.0063133654, 0.00...","[-0.063004956, 0.06692971, 0.0083840955, -0.01...",156,0.795037,0.787443,2.0
56310,2016-02-05,2016,2.0,5,Gina Cherelus,Huge construction crane collapses in Manhattan...,NEW YORK (Reuters) - A construction crane coll...,http://www.reuters.com/article/us-new-york-cra...,U.S.,Reuters,"[-0.03524109, 0.04214114, 0.004278114, 0.01927...","[-0.09076458, 0.099296205, 0.01780003, 0.03475...",90,0.206218,0.210751,5.0
56311,2016-02-05,2016,2.0,5,Eric Auchard,Showdown in Europe over privacy has U.S. firms...,FRANKFURT (Reuters) - The free flow of data ac...,http://www.reuters.com/article/us-eu-dataprote...,Technology News,Reuters,"[0.007596135, 0.047344238, -0.016574496, -0.02...","[-0.0053520403, 0.12866594, -0.031379625, -0.0...",42,0.433305,0.495872,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58234,2016-02-05 22:51:38,2016,2.0,5,Margalit Fox,"Lorna Jorgenson Wendt, 72, Defender of Rights ...","Lorna Jorgenson Wendt, whose high-stakes, high...",http://www.nytimes.com/2016/02/06/business/lor...,business,The New York Times,"[0.012283121, 0.0434221, 0.0034198314, 0.01591...","[0.021004172, 0.06160183, 0.028830199, 0.03607...",62,0.000000,0.000000,1.0
58235,2016-02-05 22:52:12,2016,2.0,5,Richard Goldstein,"Edgar D. Mitchell, Sixth Moonwalking Astronaut...","Edgar D. Mitchell, who became the sixth man to...",http://www.nytimes.com/2016/02/06/science/spac...,science,The New York Times,"[0.025910346, 0.012372937, 0.0043746694, 0.000...","[0.09182239, 0.023748346, 0.0048715225, -0.003...",194,0.818824,0.821897,2.0
58238,2016-02-05 22:59:22,2016,2.0,5,Andrew Keh,"Without Carmelo Anthony, Knicks Put Up a Valia...","Grizzlies 91, Knicks 85 Carmelo Anthony was hu...",http://www.nytimes.com/2016/02/06/sports/baske...,sports,The New York Times,"[-0.017951297, -0.007799456, -0.008263467, -0....","[-0.06582429, -0.008099353, 0.02090188, 0.0055...",157,0.000000,0.000000,1.0
58241,2016-02-05 23:10:00,2016,2.0,5,Mike Piellucci,How Troy University Became An Unlikely Breedin...,This feature is part of Super Bowl Week at VIC...,https://www.vice.com/en_us/article/aebdaj/how-...,Sports,Vice,"[0.00554524, 0.022260442, -0.00047763623, -0.0...","[-0.021131571, 0.031856976, -0.027030421, -0.0...",28,0.562409,0.535483,2.0


In [30]:
u_clusters_df= filtered_df.sort_values(by=["silhouette_values"], ascending=False)
u_clusters_df

,date,year,month,day,author,title,article,url,section,publication,embedding,sentence_embeddings,cluster,silhouette_values,cluster_mean_silhouette,cluster_size
56338,2016-02-05,2016,2.0,5,Elizabeth Pineau,"French PM defends emergency rule, says terror ...",PARIS (Reuters) - Thousands of house searches ...,http://www.reuters.com/article/us-france-shoot...,World News,Reuters,"[-0.0010319168, 0.01811194, 0.0034520088, 0.00...","[-0.039418265, 0.040478386, -0.020535825, 0.00...",697,0.972149,0.972148,2.0
57842,2016-02-05 12:59:00,2016,2.0,5,Elizabeth Pineau,Manuel Valls warns terrorist threat is here to...,By Elizabeth Pineau PARIS (Reuters) - Thousand...,https://www.businessinsider.com/r-french-pm-de...,None,Business Insider,"[-0.006441375, 0.021218868, 0.0023993433, 0.01...","[-0.036571607, 0.042007014, -0.0184135, 0.0045...",697,0.972148,0.972148,2.0
57601,2016-02-05 00:00:00,2016,2.0,5,Nicholas Bakalar,Whooping Cough Booster Shot May Offer Only Sho...,The rapidly fading effectiveness of the pertus...,http://well.blogs.nytimes.com/2016/02/05/whoop...,whooping-cough-booster-shot-vaccine,The New York Times,"[0.022979714, 0.030509854, -0.00025710632, -0....","[-0.002007108, 0.04872948, 0.00539755, -0.0385...",119,0.861531,0.859937,2.0
57378,2016-02-05 00:00:00,2016,2.0,5,"By Carina Storrs, Special to CNN",Whooping cough protection fades fast after Tda...,"(CNN)A booster vaccine for whooping cough, wh...",https://www.cnn.com/2016/02/05/health/tdap-who...,health,CNN,"[0.00059322466, 0.019088697, -0.0008608521, -0...","[0.026391841, 0.08937711, 0.02486107, 0.012044...",119,0.858342,0.859937,2.0
57166,2016-02-05 00:00:00,2016,2.0,5,Loren Grush,Morocco turns on what will become the world’s ...,Morocco has turned on its massive solar power ...,https://www.theverge.com/2016/2/5/10923000/mor...,Science,The Verge,"[0.0037360634, -0.012095271, -0.014106788, 0.0...","[0.02697129, -0.08763455, 0.000941774, -0.0067...",131,0.856667,0.845097,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56443,2016-02-05,2016,2.0,5,Christopher Miller,Tens of thousands of Syrians flee Aleppo as go...,"Backed by hundreds of Russian airstrikes, the ...",https://mashable.com/2016/02/05/syrians-flee-a...,None,Mashable,"[-0.0010621028, -0.011652682, 9.805585e-07, 0....","[-0.022732034, -0.008685973, 0.016100349, -0.0...",50,-0.042490,0.133833,6.0
56820,2016-02-05,2016,2.0,5,Jeet Heer,Bill Clinton’s Legacy Haunts the Democratic De...,Early in Thursday night’s Democratic debate in...,https://newrepublic.com/article/129261/bill-cl...,None,New Republic,"[-0.026649028, 0.061981715, 0.0025293906, 0.02...","[-0.05687423, 0.061007593, -0.0036071476, 0.02...",12,-0.078933,0.075714,20.0
58210,2016-02-05 21:41:05,2016,2.0,5,Elizabeth Williamson,Opinion | The Sincerest Form of Flattery,Editorial Observer “Immature poets imitate; ma...,http://www.nytimes.com/2016/02/06/opinion/the-...,opinion,The New York Times,"[-0.02283644, 0.07578412, 0.0036969106, 0.0350...","[0.05221253, 0.057892874, -0.006727831, 0.0416...",12,-0.112205,0.075714,20.0
56936,2016-02-05,2016,2.0,5,Cristina Marcos and Alexander Bolton,Democratic establishment starts to gang up on ...,Democratic lawmakers on both sides of the Capi...,https://thehill.com/homenews/campaign/268325-d...,None,The Hill,"[-0.013681835, 0.07243984, 0.009432732, 0.0087...","[-0.0463246, 0.05994051, 0.017483376, 0.005357...",12,-0.134094,0.075714,20.0


# Sumaryzacja

Proces ten obejmuje:
1. Utworzenie funkcji `compute_summary`, która na podstawie artykułu wylicza jego podsumowanie. Źródło: (https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/text-summarization/README.md)
2. Utworzenie funkcji `get_closest_to_center_title`, która na podstawie klastra wybiera artykuł, który jest najbliżej środka klastra
3. Utworzenie funkcji `filter_cluster`, która na podstawie klastra wybiera artykuł, który jest najbliżej środka klastra oraz usuwa artykuły, które są z tego samego źródła
3. Utworzenie listy tematów, która zawiera tytuł klastra oraz listę artykułów w tym klastrze wraz z ich podsumowaniami
4. Utworzenie raportu w formacie markdown

In [31]:
TOPICS_TO_SHOW = 10

In [32]:
"""
LexRank implementation
Source: https://github.com/crabcamp/lexrank/tree/dev
"""

import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.special import softmax
import logging

logger = logging.getLogger(__name__)


def degree_centrality_scores(
    similarity_matrix,
    threshold=None,
    increase_power=True,
):
    if not (threshold is None or isinstance(threshold, float) and 0 <= threshold < 1):
        raise ValueError(
            "'threshold' should be a floating-point number " "from the interval [0, 1) or None",
        )

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(
            similarity_matrix,
            threshold,
        )

    scores = stationary_distribution(
        markov_matrix,
        increase_power=increase_power,
        normalized=False,
    )

    return scores


def _power_method(transition_matrix, increase_power=True, max_iter=10000):
    eigenvector = np.ones(len(transition_matrix))

    if len(eigenvector) == 1:
        return eigenvector

    transition = transition_matrix.transpose()

    for _ in range(max_iter):
        eigenvector_next = np.dot(transition, eigenvector)

        if np.allclose(eigenvector_next, eigenvector):
            return eigenvector_next

        eigenvector = eigenvector_next

        if increase_power:
            transition = np.dot(transition, transition)

    logger.warning("Maximum number of iterations for power method exceeded without convergence!")
    return eigenvector_next


def connected_nodes(matrix):
    _, labels = connected_components(matrix)

    groups = []

    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)

    return groups


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError("'weights_matrix' should be square")

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    if np.min(weights_matrix) <= 0:
        return softmax(weights_matrix, axis=1)

    return weights_matrix / row_sum


def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = np.zeros(weights_matrix.shape)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1

    return create_markov_matrix(discrete_weights_matrix)


def stationary_distribution(
    transition_matrix,
    increase_power=True,
    normalized=True,
):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError("'transition_matrix' should be square")

    distribution = np.zeros(n_1)

    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        distribution[group] = eigenvector

    if normalized:
        distribution /= n_1

    return distribution

In [33]:
def compute_summary(document, num_sentences=5):
    sentences = nltk.sent_tokenize(document)
    sentences = [s[1:] if s.startswith('>') else s for s in sentences]
    # drop sentences that are duplicates
    sentences = list(dict.fromkeys(sentences))
    embeddings = model.encode(sentences, convert_to_tensor=True)
    cos_scores = util.cos_sim(embeddings, embeddings).cpu().numpy()
    centrality_scores = degree_centrality_scores(cos_scores, threshold=None)
    most_central_sentence_indices = np.argsort(-centrality_scores)
    return "\n".join(np.array(sentences)[most_central_sentence_indices[:num_sentences]])

In [34]:
def get_closest_to_center_title(df, cluster_id):
    cluster_df = df[df["cluster"] == cluster_id]
    cluster_center = np.mean(np.vstack(cluster_df["embedding"].values), axis=0)
    distanceT = cluster_df.apply(lambda row: distance.cosine(row["embedding"], cluster_center), axis=1)
    closest_record = cluster_df.loc[distanceT.idxmin()]
    return closest_record["title"]

In [35]:
def get_closest_to_center_title_by_s(df, cluster_id):
    cluster_df = df[df["cluster_mean_silhouette"] == cluster_id]
    cluster_center = np.mean(np.vstack(cluster_df["embedding"].values), axis=0)
    distanceT = cluster_df.apply(lambda row: distance.cosine(row["embedding"], cluster_center), axis=1)
    closest_record = cluster_df.loc[distanceT.idxmin()]
    return closest_record["title"]

In [36]:
def filter_cluster(df, cluster_id):
    cluster_df = df[df["cluster"] == cluster_id]
    cluster_center = np.mean(np.vstack(cluster_df["embedding"].values), axis=0)
    cluster_df['distance_to_center'] = cluster_df.apply(lambda row: distance.cosine(row["embedding"], cluster_center), axis=1)
    filtered_cluster_df = cluster_df.loc[cluster_df.groupby("publication")["distance_to_center"].idxmin()]

    return filtered_cluster_df

In [37]:
def filter_cluster_by_s(df, cluster_id):
    cluster_df = df[df["cluster_mean_silhouette"] == cluster_id]
    cluster_center = np.mean(np.vstack(cluster_df["embedding"].values), axis=0)
    cluster_df['distance_to_center'] = cluster_df.apply(lambda row: distance.cosine(row["embedding"], cluster_center), axis=1)
    filtered_cluster_df = cluster_df.loc[cluster_df.groupby("publication")["distance_to_center"].idxmin()]

    return filtered_cluster_df

In [38]:
def process_title(title):
    stripped = title.strip()
    
    hill_ending = "| TheHill"
    if stripped.endswith(hill_ending):
        return stripped[:-len(hill_ending)]
    
    return stripped

In [ ]:
#wyswietlanie wg kolejnosci najwiekszych klastrow

# make this topics array for top 10 clusters
topics1 = []
# order clusters by number of articles
unique_clusters = filtered_df["cluster"].value_counts().index

# order clusters by silhouette score
# #unique_clusters = filtered_df["cluster_mean_silhouette"].value_counts().index

for cluster_id in unique_clusters[:TOPICS_TO_SHOW]:
    example_cluster = filter_cluster(filtered_df, cluster_id)
    cluster_title = get_closest_to_center_title(filtered_df, cluster_id)
    example_cluster["title"] = example_cluster["title"].apply(process_title)
    example_cluster["summary"] = example_cluster["article"].apply(compute_summary)
    topics1.append(
        {
            "title": cluster_title,
            "articles": example_cluster[["title", "summary", "cluster", "publication","silhouette_values"]].to_dict(
                orient="records"
            ),
        }
    )

In [ ]:
#wyswietlanie wg kolejnosci najlepszych klastrow

# make this topics array for top 10 clusters
topics2 = []
# order clusters by silhouette score
unique_clusters = filtered_df["cluster_mean_silhouette"].value_counts().sort_index(ascending=False).index

for cluster_id in unique_clusters[:TOPICS_TO_SHOW]:
    example_cluster = filter_cluster_by_s(filtered_df, cluster_id)
    cluster_title = get_closest_to_center_title_by_s(filtered_df, cluster_id)
    example_cluster["title"] = example_cluster["title"].apply(process_title)
    example_cluster["summary"] = example_cluster["article"].apply(compute_summary)
    topics2.append(
        {
            "title": cluster_title,
            "articles": example_cluster[["title", "summary", "cluster", "publication","silhouette_values"]].to_dict(
                orient="records"
            ),
        }
    )

In [41]:
min_date = pd.to_datetime(filtered_df["date"], format="mixed").min()
max_date = pd.to_datetime(filtered_df["date"], format="mixed").max()

In [42]:
%load_ext pretty_jupyter

In [43]:
%%jinja markdown
# News summarization

### Dates covered: {{ min_date.strftime("%Y-%m-%d") }} - {{ max_date.strftime("%Y-%m-%d")}} 

### Number of topics: {{ unique_clusters | length }}

### Total Number of articles: {{ filtered_df | length }}

## Summaries (by the biggest clusters):
{% for topic in topics1[:TOPICS_TO_SHOW] %}
----
### {{loop.index}}. {{ topic['title'] }} ({{ topic['articles'] | length }} articles)
{% for article in topic['articles'] %}
#### {{ article['publication'] }}: {{ article['title'] }} 
{{ article['summary'] }}


**Silhouette Value:** {{ article['silhouette_values'] }}
{% endfor %}
{% endfor %}

## Summaries (by the best clusters):
{% for topic in topics2[:TOPICS_TO_SHOW] %}
----
### {{loop.index}}. {{ topic['title'] }} ({{ topic['articles'] | length }} articles)
{% for article in topic['articles'] %}
#### {{ article['publication'] }}: {{ article['title'] }}
{{ article['summary'] }}


**Silhouette Value:** {{ article['silhouette_values'] }}
{% endfor %}
{% endfor %}


# News summarization

### Dates covered: 2016-02-05 - 2016-02-05 

### Number of topics: 109

### Total Number of articles: 359

## Summaries (by the biggest clusters):

----
### 1. Democratic debate: 6 takeaways (10 articles)

#### CNN: Democratic debate: 6 takeaways 
Here are the takeaways from the intense MSNBC Democratic debate: Clinton goes after Sanders for his 'artful smear' campaign In her second presidential campaign, Clinton clearly doesn't want to win through a war of attrition.
There's a reason Clinton's campaign decided to embrace four more Democratic debates: She's a bare-knuckle brawler, and she's not going to lose the nomination because she wasn't willing to hit Sanders hard enough.
She finally found an answer to Sanders' criticism: "A vote in 2002 is not a plan to defeat ISIS."
And she follows with the definition of 'progressive' It's been a struggle for months for Clinton to find a way to tell Sanders' supporters that she's on the same page with them policy-wise -- it's just that their big ideas could never, ever make it through Congress.
Clinton to Sanders: 'If you have something to say, say it' "Time and time again, by innuendo, by insinuation, there is this attack that he is putting forth which really comes down to, you know, anybody who ever took donations or speaking fees from any interest group has to be bought.


**Silhouette Value:** 0.2210465967655182

#### Mashable: Hillary Clinton accuses Bernie Sanders of 'very artful smear' 
Clinton confronted Sanders, saying "time and time again, by innuendo, by insinuation" he attacks any candidate that has ever taken money from an interest group.
:fire::fire::fire: #DemDebate pic.twitter.com/Dns20rfXBW — Mashable News (@MashableNews) February 5, 2016 Sanders started a feud this week when he questioned Clinton's progressive credentials.
WASHINGTON — Hillary Clinton and Bernie Sanders took direct shots at each other in a one-on-one matchup for the first time in a debate on Thursday, and Wall Street continued to be their biggest source of friction.
And said called Clinton's strong support from a long list of Democratic elected officials made her part of the "establishment" he seeks to fight against.
Way to make it awkward, @chucktodd #DemDebate pic.twitter.com/DAeSeXnuVb — Mashable News (@MashableNews) February 5, 2016 Still, despite the heated moments, the debate ended on a more friendly note, when both Clinton and Sanders expressed respect for one another.


**Silhouette Value:** 0.20372642576694489

#### New Republic: That’s it for the fifth Democratic debate. Here’s what you missed. 
Gwyneth Kelly notes that Clinton made an interesting argument for why she’s not the establishment candidate.
Alex Shephard notes that Clinton is still running on her old tagline, “Who do you want to answer the phone at 3 a.m.?”  Elspeth Reeve admires MSNBC’s use of a split screen.
Elizabeth Bruenig argues that Hillary Clinton wants you to believe she’s tough on Wall Street, even when she’s not.
Clinton also called the Sanders campaign’s attacks on her Wall Street ties an “artful smear,” Foehringer Merchant writes.
Shephard says Clinton still hasn’t figured out how to answer questions about her Wall Street connections.


**Silhouette Value:** 0.16444888710975647

#### People: Hillary Clinton Goes Head-to-Head with Bernie Sanders in Democratic Debate 
Although the two did tackle their respective plans for several issues, it seems Clinton set her sights on calling Sanders out for his months of criticism – or “artful smear,” as Clinton put it – suggesting hefty speaking fees from big business have influenced Clinton’s campaign.
“There is a reason why these people are putting huge amounts of money into your political system,” he said, noting that Clinton’s alleged conduct is “undermining American democracy.”   Sanders walked in to the debate with a 20-point lead over Clinton, according to an NBC-Wall Street Journal-Marist poll.
“So I think it’s time to end the very artful smear that you and your campaign have been carrying out.”    Sanders held to his familiar critique, noting that a super PAC funding Clinton is partly funded by banks.
Despite rejecting Sanders’ criticisms, Clinton did note that she had not “done the job I should in explaining my record” in regard to her financial regulation.
“[Senator Bernie Sanders] has said he wants to run a positive campaign.


**Silhouette Value:** 0.12284929305315018

#### Politico: Insiders: Bernie scored in debate with Wall Street slams 
It was particularly brutal when she ticked off the various votes he's made over the years on guns and other issues which are decidedly not progressive.” And a third New Hampshire Democrat said Clinton “hammered home the point that she is the progressive who can get things done.” But some disagreed, arguing that Sanders was steady and believable during this discussion, and Clinton seemed less authentic.
“All her responses fail to adequately address the issue.” “Hillary had an okay answer,” another Nevada Democrat agreed, “but you can tell she still looks uncomfortable delivering it.” Some Democratic insiders said while that part of the debate was fought on Sanders’ terrain, he failed to make the connection that Clinton would do the bidding of the moneyed interests he rails against.
She should have hit harder, however, and demanded Bernie outright say he thinks she can be bought, rather than letting him continue to insinuate it on the margins.” Other key takeaways from the fifth Democratic primary debate Thursday night: Clinton dinged Sanders during the blockbuster first segment.
“His attack falls apart when you discuss it beyond sound bites.” “MSNBC was transparently trying to get Bernie and Hillary to attack each other the way Republican candidates do,” a Nevada Democrat added.
It's not rocket science.” Some Democrats admitted that Sanders’ answer to the electability question was affecting their ability to support the Vermont senator.


**Silhouette Value:** 0.03962671011686325

#### The Hill: Clinton, Sanders tangle in heated debate  
"I think it's time to end the very artful smear that you and your campaign have been carrying out over the past few weeks, and let’s talk about the issues that divide us.”  But Sanders kept the pressure up on Clinton's ties to Wall Street when he suggested she undervalued the role of money in politics.
“If we’re going to talk progressive, I don’t think it was progressive to vote against the Brady Bill five times,” Clinton said.
Clinton came out swinging on Thursday, arguing that the Vermont Independent's ideas are unachievable because “the numbers just don’t add up” in Sanders’s proposals, adding, "I'm not making promises that I cannot keep."
And during a heated exchanged that elicited boos from the audience, she pushed back at Sanders's insinuation on the stump that she's in the pocket of Wall Street because she has taken donations from the financial industry.
An aggressive Clinton dominated early portions of the MSNBC/New Hampshire Union Leader debate, while Sanders later struck back on banking and campaign finance as the two squared off in the first one-on-one debate of 2016 just five days before the Granite State goes to the polls.


**Silhouette Value:** 0.20706993341445923

#### The New York Times: Who Won the Debate? Critics Are as Split as the Candidates 
This was the most intense debate of the entire cycle, possibly foreshadowing an epic, long-running series of face-to-face contests alternating with primaries and caucuses well into the spring.” —Mark Halperin, managing editor of Bloomberg Politics “Now if they could just split the gig -- Bernie would cover domestic, Hillary on foreign policy.” — Mia Farrow, actress and activist video
“The candidate that will gain from this zero-sum debate — the winner — will turn on something of an x-factor: did the undecided Democrats watching prefer Hillary Clinton’s direct, aggressive attacks on Bernie Sanders, most of them early in the night?
There’s an undeniable decency to him that you don’t often see at this level of American politics.” — Ezra Klein, founder of Vox.com “Crucial distinction between Bernie and Hillary on Flint: She suggests solutions, he demands punishments.” — Charlotte Alter, writer for Time “A weird advantage Sanders has in things like this: The naturalness in his interactions accentuates how Clinton struggles to seem at ease.” — Philip Bump, writer for The Washington Post’s blog “The Fix” “No offense to Martin O’Malley but it was really refreshing to finally get to hear Clinton and Sanders one on one.” — Joy Reid, author of “Fracture: Barack Obama, the Clintons, and the Racial Divide” “No surprise, Clinton won Foreign Policy.
Sanders wins round on trade agreements with Democrats.” — Steve Forbes, chairman of Forbes Media “The national security portion of the debate couldn’t end soon enough for Bernie Sanders.
This is why Clinton’s funding is so central to this campaign.” — Marc Cooper, retired professor at USC Annenberg School for Communication and Journalism “Both made strong attacks, and both defended effectively.


**Silhouette Value:** 0.010576815344393253

#### Vice: Hillary Clinton and Bernie Sanders Are Fighting Over the Future of the Democratic Party 
She went on to lambast what she described as the Sanders campaign's "innuendos" and "insinuation" that her political career has been bought and sold by special interests.
The exchange was representative of the broader conflict going on between the moderate and progressive factions of the political left in the US—a divide that, whether or not the Democrats win in November, will likely determine the future of the party for years to come.
Toward the end of the debate, the moderators touched on foreign policy, but even then, the drama was limited to Sanders's continued attacks on Clinton's vote for the Iraq War, and his otherwise rambling responses on how he would handle international affairs as president.
For two hours straight, it was just Clinton and Sanders on stage in New Hampshire, and almost immediately MSNBC moderators Rachel Maddow and Chuck Todd started asking different variations of the question that Democratic voters have been fighting over for months: What does it mean to be a true progressive?
For weeks now, Vermont Senator Bernie Sanders and former Secretary of State Hillary Clinton have been quietly ripping into each other.


**Silhouette Value:** 0.15586675703525543

#### Vice News: Underdog Clinton Goes After Sanders in New Hampshire Debate 
Hillary Clinton's rising frustrations at Bernie Sanders's surging campaign burst onto the debate stage last night in New Hampshire, as she accused her Democratic nomination rival of inciting a "smear campaign" against her, despite his insistence he would wage a clean, fair fight and not go negative.
Clinton challenged Sanders on the authenticity of his campaign and on-stage assertions that he respects the former secretary of state "very much."
While the candidates debated, the Sanders campaign sent an email to reporters titled "Hillary Clinton — a progressive on some days," which included a list of issues on which the former secretary of state has taken a more moderate position.
In the days of campaigning ahead of the New Hampshire primary on Tuesday, Clinton will have to balance an effective takedown of what she perceives as Sanders's weak points on foreign policy and unachievable goals on healthcare and other proposals, while not going so far as to alienate his support base, in the event she does take the nomination.
Clinton accused Sanders of playing "gatekeeper" on progressivism, arguing that even President Obama wouldn't fit Sanders' definition.


**Silhouette Value:** 0.18667525053024292

#### Vox: Finally, Bernie Sanders and Hillary Clinton had a real argument over how politics works 
And where Clinton's experience gives her deep knowledge of virtually every facet of American policymaking, Sanders's career has let him focus on the issues he cares about, and left him poorly informed on international affairs.
Amidst a question about President Obama's escalation of the war against ISIS, Sanders skipped past the specifics, saying simply, "Let me agree with much of what the secretary said," and then pivoted to ground he's more comfortable on — attacking her 2002 vote to authorize the Iraq War.
It takes nothing away from the historic nature of her candidacy to say that everyone would characterize Clinton — a former first lady, senator, secretary of state, and Democratic frontrunner — as exemplifying the establishment.
Her view of the political system is realistic, her knowledge of the issues is deep, and her social ties are strong.
Sanders's disinterest led to Clinton's best line of the night.


**Silhouette Value:** 0.14666828513145447


----
### 2. Twitter Steps Up Efforts to Thwart Terrorists’ Tweets (10 articles)

#### Gizmodo: Twitter Says It's Shut Down 125,000 Terrorist Accounts Since May 
These “strategies” seem out-of-touch and straight out of the PR playbook, but canning 125,000 Twitter accounts is still impressive: The Brookings Institute recently estimated that only 90,000 terrorist accounts existed worldwide.Twitter reminds us there’s no “magic algorithm” to completely halting extremist activity online, and it often involves making judgment calls based on very little information.
But in a blog post today, Twitter says that kind of behavior flagrantly violates its terms of service, and reports that it’s suspended tens of thousands pro-ISIS accounts since May.“We have increased the size of the teams that review reports, reducing our response time significantly,” the company states.
Terrorist groups like ISIS use social media to rally support around the world, using sites like Twitter to mobilize sympathizers into possible plots.
“We also look into other accounts similar to those reported and leverage proprietary spam-fighting tools to surface other potentially violating accounts for review by our agents.”Tech companies like Twitter and Facebook are in a tough position—there’s an obvious problem, and they want to look like they’re doing something.
And Facebook COO Sheryl Sandberg went so far last month as to say “likes” somehow discredit terrorist propaganda on Facebook.


**Silhouette Value:** 0.7533852458000183

#### Mashable: Twitter suspends more than 125,000 accounts for promoting terrorism 
Most of those accounts were tweeting about the Islamic State, an extremist group that has taken over a swath of territory in Iraq and Syria and committed other acts of terrorism outside those nations.
As Twitter said in its announcement, there's no "magic algorithm" for finding extremist content on the Internet.
Twitter also says it has built up the team that reviews reports of terrorist threats and promotion, and has seen "this type of activity shifting off of Twitter," the company wrote in a blog post.
And of course, even if Twitter is becoming a less friendly space for ISIS sympathizers, it's not like they're fleeing the Internet.
The idea was that the more these extremists tweeted, the more U.S. officials would know about them and their followers.


**Silhouette Value:** 0.704115629196167

#### Politico: Twitter says it’s acting against terror groups 
As Washington presses the tech industry to do more to combat online extremism, Twitter took to its blog to detail how it has ramped up its efforts in recent months, stressing it has "already seen results, including an increase in account suspensions and this type of activity shifting off of Twitter."
(Twitter also joined a high-profile session with Obama administration officials in Silicon Valley last month, POLITICO previously reported.)
Twitter has suspended more than 125,000 accounts for "threatening or promoting terrorist attacks primarily related to ISIS" since the middle of last year, the company announced today.
"As an open platform for expression, we have always sought to strike a balance between the enforcement of our own Twitter Rules covering prohibited behaviors, the legitimate needs of law enforcement, and the ability of users to share their views freely - including views that some people may disagree with or find offensive," Twitter said.
And it pointed out its work with organizations to counter extremist content and its attendance at government-run meetings, including the White House's summit on the issue last year.


**Silhouette Value:** 0.668079674243927

#### Reuters: Twitter suspends over 125,000 accounts for 'promoting terrorist acts' 
(bit.ly/1KvrmZ5) Twitter’s announcement comes as many tech companies – led by Facebook – have taken stronger steps to police controversial content online in the face of threats from legislators to force the companies to report “terrorist activity” on their sites to law enforcement.
The announcement was also notable because Twitter has said little about its efforts to combat Islamic State, also known as ISIS, and similar groups even though it has been criticized for not doing enough.
Seamus Hughes, deputy director of George Washington University’s program on extremism, said Friday’s report showcased an “impressive number” of takedowns, but said that Twitter still appears to police extremist content in a mostly “episodic” way.
“Addressing the use of social media by terrorists will require a sustained and cooperative effort between the technology sector, the Intelligence Community, and law enforcement,” he said.
WASHINGTON (Reuters) - Twitter Inc (TWTR.N) has shut down more than 125,000 terrorism-related accounts since the middle of 2015, most of them linked to the Islamic State group, the company said in a blog post on Friday.


**Silhouette Value:** 0.7596532702445984

#### The Hill: Twitter suspends more than 125K accounts for promoting terrorism  
The announcement comes at a time when Twitter and other social media companies are under intense pressure from Washington to blunt the online influence of terror organizations like the Islamic State in Iraq and Syria (ISIS).
Shortly after Twitter’s post went live, Brett McGurk, who lead’s the State Department’s effort to build support against ISIS, applauded Twitter’s “Welcome moves.” And Rep. Adam Schiff (Calif.), ranking Democrat on the House Intelligence Committee, called Twitter’s removals “an important part of the fight against radical islam.” View the discussion thread.
In April of last year, Twitter explicitly banned statements “threatening or promoting” terrorism.
“As many experts and other companies have noted, there is no ‘magic algorithm’ for identifying terrorist content on the internet, so global online platforms are forced to make challenging judgement calls based on very limited information and guidance,” Twitter said in its blog.
Social media companies have been more aggressive in publicizing their anti-terror efforts since that meeting.


**Silhouette Value:** 0.7117176651954651

#### The New York Times: Twitter Steps Up Efforts to Thwart Terrorists’ Tweets 
“As the nature of the terrorist threat has changed, so has our ongoing work in this area,” Twitter said in a statement, adding that it “condemns the use of Twitter to promote terrorism.” The company said its collective moves had already produced results, “including an increase in account suspensions and this type of activity shifting off Twitter.” The disclosure follows intensifying pressure on Twitter and other technology companies from the White House, presidential candidates like Hillary Clinton and government agencies to take more action to combat the digital practices of terrorist groups.
“Twitter has ramped up its response significantly since June 2014, and again since the Paris attacks, but ISIS has still maintained a notable presence on the platform,” said J. M. Berger, a fellow with George Washington University’s Program on Extremism and author of books on extremist issues.
He added that Facebook, the biggest social network, has also “been very aggressive on this issue, about as much as anyone could reasonably expect, but we still see some ISIS activity.” Twitter’s disclosure of the number of terrorist account suspensions sets it apart from its social media peers.
And for years, extremist groups like the Islamic State have taken advantage of that stance, using Twitter as a place to spread their messages.
Online extremist content includes Twitter accounts like @Musliiiiimah_1.


**Silhouette Value:** 0.803056001663208

#### The Verge: Twitter has suspended 125,000 ISIS accounts since mid-2015 
The move comes as Twitter is facing increasing criticism from some in Washington for the perceived presence of ISIS-friendly Twitter accounts.
Twitter claims those tactics have resulted in more than 125,000 accounts suspended for threatening or promoting terrors acts since the middle of last year.
At the same time, Twitter has also drawn criticism for being overly aggressive in suspending accounts, as when it briefly suspended an Arab Spring activist over a misunderstanding at the end of last year.
Twitter is coming clean about its efforts to fight terrorist-aligned accounts.
"We have already seen results," the post reads, "including an increase in account suspensions and this type of activity shifting off of Twitter."


**Silhouette Value:** 0.758265495300293

#### Vice: ​Twitter: We've Blocked 125,000 ISIS Accounts Since Mid-2015 
Twitter has long come under fire for ISIS's extensive presence on the platform.
On top of that, Twitter has added more personnel to look at any accounts reported for threatening or promoting terrorism, and is also "leverag[ing] proprietary spam-fighting tools to surface other potentially violating accounts for review by our agents."
As the company itself points out, in July 2015, FBI Director James Comey said that Twitter was "very good and thoughtful and hardworking at trying to shut down [terrorism-related] accounts."
Twitter is also partnering with anti-extremist organizations, and has been cooperating with law enforcement "when appropriate."
Twitter's latest announcement comes at a time when ISIS's online presence is increasingly politicized.


**Silhouette Value:** 0.7638776898384094

#### Vox: Twitter Says It Took Down 125,000 ISIS-Related Accounts Since Last Year 
Twitter does not actively look for terrorist accounts, but instead relies on users to report them.
Twitter says it is doing its part to combat terrorism online.
A study released in the fall by George Washington University found that Twitter was indeed the platform of choice for ISIS propagandists in part because creating accounts is so simple.
The company announced today that it has “suspended over 125,000 accounts for threatening or promoting terrorist acts primarily related to ISIS” since the middle of last year.
This is one of the challenges for Twitter, which offers its users anonymity (unlike, say, Facebook).


**Silhouette Value:** 0.7464978694915771

#### Wired: Twitter Wants You to Know That It Is Fighting Terrorists 
While Facebook has taken a blunt approach toward removing any and all posts that have even a whiff of terrorism, Twitter has sought to strike a balance between protecting free speech and cracking down on people who use Twitter as a way to promote violence or threats.
Twitter wants you to know that it is just as horrified as you are about the tweets on its platform promoting terrorism—and that, yes, the company is trying to combat it.
In recent months, Twitter has come under fire as ISIS propagandists used the platform to promote their cause and recruit would-be militants.
People complain readily (and rightfully) about Twitter's failure to find a real solution to harassment on its platform.
In some cases, after all, one person's political speech is another's call for terrorism.


**Silhouette Value:** 0.662918210029602


----
### 3. Why the UN ruled Julian Assange is being “arbitrarily detained," and why it matters (10 articles)

#### Gizmodo: UN Panel Rules That, Yes, Julian Assange Has Been Arbitrarily Detained 
A United Nations panel has ruled that Julian Assange has been “arbitrarily detained by the Governments of Sweden and the United Kingdom” and believes he is “entitled to his freedom of movement.”Assange and his legal team complained to the UN’s Working Group on Arbitrary Detention in 2014 that his living in 300 square feet within the Ecuadorian Embassy was unwarranted and had taken a toll on his health.
The Working Group also considered that the detention should be brought to an end and that Mr. Assange should be afforded the right to compensation.Swedish authorities have been seeking Assange over allegations of rape, but he has resisted questioning—largely over fears that he may end up extradited to the US.
Now, the panel has determined that Assange has “been subjected to different forms of deprivation of liberty” since his initial arrest back in 2010.
Indeed, London’s police force has already pointed out that the UN ruling doesn’t undermine the European arrest warrant against him, and say he’d still be taken into custody if he were to leave the embassy.According to BBC Radio 4, Assange will provide a reaction to the news via Skype later today.
It explains:The Working Group therefore requested Sweden and the United Kingdom to assess the situation of Mr. Assange to ensure his safety and physical integrity, to facilitate the exercise of his right to freedom of movement in an expedient manner, and to ensure the full enjoyment of his rights guaranteed by the international norms on detention.


**Silhouette Value:** 0.7771936058998108

#### Mashable: UN: Julian Assange should go free and be paid for years-long standoff 
UPDATED: 4:40 p.m. GMT LONDON — WikiLeaks founder Julian Assange said a UN decision that he should be allowed to move freely without arrest was a "vindication," and he criticised the Swedish and British governments for their objection to the decision.
Foreign Secretary Philip Hammond said the decision was "ridiculous" and that Assange was a "fugitive from justice."
Front row seats #Assange #UNWGAD pic.twitter.com/dmOIcAeQas — Chris Ellis (@MrChrisEllis) February 5, 2016 The panel's decision doesn't legally require the governments to lift or not serve the arrest warrant on Assange, but it would support his case for being allowed to move freely.
Early Thursday morning, Assange issued a statement via the WikiLeaks Twitter account that if the UN panel ruled against him, he would walk out of the embassy at noon Friday and accept arrest.
The United Nations Working Group on Arbitrary Detention said earlier Friday that Assange has been arbitrarily held since his arrest in London in Dec. 7, 2010, over allegations of sex crimes.


**Silhouette Value:** 0.8234489560127258

#### People: Julian Assange Makes Appearance in London After U.N. Decision 
On Friday, the journalist and publisher took to the small balcony of the Ecuadorian Embassy in London, where he’s claimed political asylum since 2012, to praise the United Nations for deciding he’s been unlawfully detained, with a cry of “How sweet it is!” “This is a victory that cannot be denied,” Assange added to the swelled ranks of reporters gathered underneath.
Working Group on Arbitrary Detention today ruled this detention unlawful, adding that Assange is “entitled to his freedom of movement and to compensation”.
“I have been detained now without charge in this country, the United Kingdom, for five-and-a-half-years,” Assange said to reporters via video link on Friday.
— Philip Hammond (@PhilipHammondUK) February 5, 2016  Speaking on British TV, Hammond later called the U.N.’s decision “flawed” and “ridiculous”, before adding, “He’s hiding from justice in the Ecuadorian embassy.
In a tweet, British Foreign Secretary Philip Hammond described Assange as a “fugitive from justice, voluntarily hiding in the Ecuadorian embassy”.


**Silhouette Value:** 0.7053467631340027

#### Politico: Julian Assange: I've been vindicated 
GENEVA — A U.N. human rights panel says WikiLeaks founder Julian Assange has been "arbitrarily detained" by Britain and Sweden since December 2010.
Swedish prosecutors want to question Assange over allegations of rape stemming from a working visit he made to the country in 2010 when WikiLeaks was attracting international attention for its secret-spilling ways.
Assange has consistently denied the allegations but declined to return to Sweden to meet with prosecutors and eventually sought refuge in the Ecuadorean embassy in London, where he has lived since June 2012.
Assange says the findings are a "vindication."
Working Group on Arbitrary Detention said his detention should end and he should be entitled to compensation.


**Silhouette Value:** 0.7707222104072571

#### The Hill: UN panel concludes WikiLeaks founder has been ‘arbitrarily detained’  
A United Nations working group on Friday concluded that WikiLeaks founder Julian Assange has been “arbitrarily detained” by Sweden and the United Kingdom during his more than three-year stay in the Ecuadorian embassy in London.
“The Working Group maintains that the arbitrary detention of Mr. Assange should be brought to an end, that his physical integrity and freedom of movement be respected, and that he should be entitled to an enforceable right to compensation,” said Seong-Phil Hong, the U.N. panel’s leader.
In addition to his confinement in the embassy, the U.N. panel also raised concerns about Assange's 10-day detainment in a London prison back in 2010, and the report accused Swedish prosecutors of a lack of diligence, which led to his lengthy stay in the embassy.
Assange requested asylum in Ecuador in 2012 and has remained in the country’s embassy in London ever since.
The decision could have little effect, since British authorities have said that if Assange leaves the embassy, they still plan to arrest and extradite him to Sweden, where he is wanted in questioning for rape accusations.


**Silhouette Value:** 0.8011159896850586

#### The New York Times: Australia Reacts to U.N. Decision on Julian Assange 
SYDNEY, Australia — Julie Bishop, Australia’s foreign minister, has sought legal advice on the implications of a United Nations decision on Friday that the WikiLeaks founder Julian Assange, an Australian, has been unlawfully held by Sweden and Britain for more than five years.
“I have now read the report and I am seeking legal advice on its implications for Mr. Assange, as an Australian citizen,” she said in a brief statement issued from London.
The lawfulness of my detention, or otherwise, is now a matter of settled law.” The United Nations Working Group on Arbitrary Detention criticized a lack of diligence in the Swedish prosecutor’s office and concluded Mr. Assange’s detention should be bought to an end.
Mr. Assange said in the video link that he understood what he called a domestic “political imperative” of both governments to look like they had made the correct decision around his detention, but went on to say that it had been unlawful.
Mr. Assange, 44, sought refuge in the Ecuadorean Embassy in London in 2012 to avoid extradition to Sweden, where he faced an allegation of rape.


**Silhouette Value:** 0.6984354257583618

#### The Verge: UN: Julian Assange has been arbitrarily detained and deserves compensation 
"To go against a decision would be to go against a unit of the United Nations, and their own interpretation of human rights," Per Samuelsson, a lawyer for Assange, told The New York Times yesterday.
In a statement released Friday, the UNWGAD said that Sweden and the UK should work to guarantee Assange's "safety and physical integrity, to facilitate the exercise of his right to freedom of movement in an expedient manner, and to ensure the full enjoyment of his rights guaranteed by the international norms on detention."
This week, Sweden said it would reject the panel's findings and continue to pursue extradition, though Assange's lawyers are hopeful that the UN decision will pressure authorities to change its position.
"While I understand the momentary political imperatives for representatives of Sweden and the UK to look like they were correct in their earlier, unlawful decisions, the fact is that they are not," Assange said.
In a report published today, the UN Working Group on Arbitrary Detention (UNWGAD) said that Sweden and the UK should compensate Assange for the detention, though the ruling is not legally binding.


**Silhouette Value:** 0.7818725109100342

#### Vice: It's Official: UN Tells UK and Sweden Julian Assange Is Arbitrarily Detained 
"The Working Group on Arbitrary Detention considers that the various forms of deprivation of liberty to which Julian Assange has been subjected constitute a form of arbitrary detention," said Seong-Phil Hong, who currently heads the expert panel, in a statement.
Anna Ekberg, spokesperson for the Swedish foreign ministry, told the Guardian on Thursday: "The UN working group on arbitrary detention has concluded that Mr Assange is arbitrarily detained.
Assange sent a complaint about his situation to the UN Working Group on Arbitrary Detention in 2014.
A UN panel has ruled that Wikileaks editor Julian Assange's confinement to the London Ecuadorian embassy qualifies as arbitrary detention.
"The Working Group maintains that the arbitrary detention of Mr. Assange should be brought to an end, that his physical integrity and freedom of movement be respected, and that he should be entitled to an enforceable right to compensation," Mr. Hong added.


**Silhouette Value:** 0.7559503316879272

#### Vice News: 'Victory and Vindication': Julian Assange Slams UK Response to UN Ruling 
Britain and Sweden have rejected the panel's opinion that Wikileaks founder Julian Assange is a victim of arbitrary detention, with the UK government saying he will be arrested if he leaves the Ecuadorean embassy where he has been living for three and a half years.
The Swedish government echoed the UK's opinion that Assange could not be a victim of arbitrary detention given that he chose to live in the embassy and was free to walk out at any time.
"Mr Assange has chosen, voluntarily, to stay at the Ecuadorian Embassy and Swedish authorities have not control over his decision to stay there," it said.
The UN Panel on Arbitrary Detention formally ruled on Friday that "the various forms of deprivation of liberty" Assange had experienced — an initial period of remand at London's Wandsworth Prison in 2010, followed by house arrest and his time at the embassy — mounted to arbitrary detention, a decision taken after more than a year considering a complaint that the computer hacker filed to the working group in September 2014.
"Mr Assange is free to leave the Embassy at at any point.


**Silhouette Value:** 0.8147341012954712

#### Vox: Why the UN ruled Julian Assange is being “arbitrarily detained," and why it matters 
The UN body "called on the Swedish and British authorities to end Mr. Assange’s deprivation of liberty, respect his physical integrity and freedom of movement, and afford him the right to compensation."
By calling the decision "legally binding" and emphasizing the authority of the UN panel, Assange is hoping both to show that his claim that he's being unlawfully detained is legitimate and to raise the stakes for the British and Swedish governments for continuing to seek his arrest and extradition.
It's not legally binding — the UK isn't now required to let him go free, and has explicitly stated that it won't — but Assange is likely hoping this will help him build a larger case for one day convincing the UK it should let him leave the embassy where he's lived since 2012.
So Correa agreed to grant Assange asylum in the Ecuadorian embassy in London so that the British couldn't arrest him and extradite him to Sweden, and potentially on to the US.
As part of this effort, Assange and his lawyers in 2014 appealed to the UN human rights body in charge of determining whether individuals imprisoned around the world are being detained lawfully, asking them to examine his case and issue a ruling.


**Silhouette Value:** 0.8187257051467896


----
### 4. National 2016 poll: Ted Cruz and Marco Rubio gain on Donald Trump (4 articles)

#### CNN: Quinnipiac poll: Sanders and Clinton neck-and-neck 
Sanders handily beats Trump by a 10-point margin, defeats Cruz by four points and ties Rubio, with each taking 43%, according to Quinnipiac.
CNN's own national Poll of Polls from just before Iowa found Clinton at 53%, Sanders at 36% and former Maryland Gov.
Trump tops the field with 31% of support, with Cruz and Rubio following at 22% and 19%, respectively, according to the Quinnipiac poll.
The poll shows Bloomberg largely cutting into Sanders' lead over Trump, bringing Sanders down to 35% to Trump's 36%, with Bloomberg taking 15%.
National polls conducted before the Iowa caucuses showed the Democratic race tightening, but with Clinton still leading Sanders by double digits.


**Silhouette Value:** 0.09880458563566208

#### Politico: National 2016 poll: Ted Cruz and Marco Rubio gain on Donald Trump 
The survey finds Trump with 31 percent nationally, Cruz with 22 percent and Rubio with 19 percent.
The picture of a neck-and-neck race is a huge change from Quinnipiac’s last national poll conducted Dec. 16-20 that showed Clinton with a massive lead over Sanders, 61 percent to 30 percent.
No other candidate polls in the double digits: Retired neurosurgeon Ben Carson has 6 percent, the rest are at or below 3 percent and 9 percent of those polled are still undecided.
Clinton leads Sanders 44 percent to 42 percent, well within the margin of error of the poll, which was conducted after the Iowa caucuses.
Cruz comes in second in that category with 15 percent, and 7 percent would not support Rubio.


**Silhouette Value:** 0.20815743505954742

#### The Hill: Poll: Trump leads NH; Rubio in second  
With four days left before the Granite State primary, an NBC/Wall Street Journal/Marist poll released Friday shows Trump leading the GOP pack, with 30 percent support, followed by Rubio, at 17.
Also gaining ground in poll: Ted Cruz, up from 12 percent to 15, and Jeb Bush, up a point to 9 percent.
In the same survey a week earlier, Trump sat at 31 percent and Rubio 11.
The NBC News poll is at least the third this week to show Rubio rising to second place behind Trump after his strong third-place finish in the Iowa caucuses.
John Kasich (10 percent) comes in fourth in the survey behind Trump, Rubio and Cruz.


**Silhouette Value:** 0.33360984921455383

#### The New York Times: Chris Christie's Support in New Hampshire Falls, Poll Says - First Draft. Political News, Now. 
An NBC/WSJ/Marist poll shows Mr. Christie’s support among likely Republican voters dropped to 4 percent, from 7 percent, in the last week, leaving him in sixth place.
Jeb Bush of Florida, 9 percent, who are banking on strong performances in New Hampshire.
That survey still showed Mr. Trump leading at 29 percent, but Mr. Rubio was just 10 points behind him.
Meanwhile, separate polls released on Friday show Mr. Trump’s support holding firm in New Hampshire.
A Suffolk University/Boston Globe poll indicated that Mr. Rubio was gaining momentum after his strong showing in Iowa.


**Silhouette Value:** 0.4147711396217346


----
### 5. Three Big Questions on the Job Market, and How January’s Numbers Answer Them (6 articles)

#### Business Insider: Barclays interest rate hike forecast 
The firm writes (emphasis added): While we have not changed our view that labor markets remain healthy and, in turn, recession risk for the US economy remains low, the weakness in services sector employment in the establishment report is likely to keep uncertainty about the state of the economy elevated.
Barclays economists got mixed signals from Friday's jobs report.
Now, we'd remind readers that after the market volatility in August, Barclays pushed out their forecast for the first rate hike in a decade to March 2016.
And so even though the headline payrolls number was lower than expectations, wage growth, the unemployment rate, and seasonal considerations made it a solid report overall.
The Fed raised rates in December 2015.


**Silhouette Value:** 0.1935020089149475

#### CNBC: Why the jobs report is actually good news—commentary 
While it may not be obvious on the surface, the January U.S. job report could provide a lift to flagging financial spirits.
The gains are smaller than we had seen in recent reports — and smaller than expected — but there was some good news in there.
But continued gains in jobs and wages should reduce the likelihood of a worst-case outcome and allow the Federal Reserve to continue normalizing conditions later on this year.
After five weeks of disappointing market outcomes, the world could use some good news.
By that measure, January's outcome is hardly disappointing.


**Silhouette Value:** 0.42617473006248474

#### Politico: Economy added 151,000 jobs in January 
Jason Furman, chairman of the White House Council of Economic Advisers, acknowledged the January slowdown, but said in a blog post that the job-growth numbers were still "well above the pace necessary to maintain a low and stable unemployment rate."
January's weak job growth, combined with last week's Commerce Department report of lackluster GDP growth during the fourth quarter of 2015, is potentially bad news for any Democratic nominee who hopes to run on President Barack Obama's economic legacy — though that nominee could still point to an unemployment rate that's been cut in half since the president took office.
Elise Gould, senior economist at the left-leaning Economic Policy Institute, said that despite January's low number the jobs report "is a sign that the economy continues to recover."
He noted that unemployment hadn't been 4.9 percent since February 2008, and that a "2.9-percent annualized pace over the past six months is the strongest since the recovery began."
The jobs report followed news last week from the Commerce Department that GDP increased 0.7 percent during the fourth quarter of 2015 — a decline from the third quarter’s 2 percent — and 2.4 percent for the year.


**Silhouette Value:** 0.43692314624786377

#### The Hill: Jobless rate falls to 4.9 percent, an 8-year low  
Economists had expected between 185,000 and 200,000 jobs, so January’s numbers will add to fears about the strength of the economy in 2016.
The U.S. economy added 151,000 jobs in January — well below the robust pace at the end of 2015, but placing the unemployment rate under 5 percent for the first time in nearly eight years.
“While 151,000 new jobs in January is below expectations and off pace from prior months, the data shows America's recovery is continuing,” Bovino said.
Sophia Koropeckyj, managing director at Moody’s Analytics, said that “January's disappointing top-line payroll number belies the fairly healthy and resilient employment situation.” “Consumer industries such as retail, leisure/hospitality and financial services continued to expand in January, underscoring the resilience of U.S. consumer to global weakness and, so far, financial turmoil," Koropeckyj said.
“Amid all the global economic turmoil and domestic market gyrations, positive job growth, the drop in the unemployment rate to 4.9 percent, and the uptick in wages show the U.S. is heading in the right direction.” Bovino said the recent jobs data will give the Federal Reserve plenty to weigh at its next meeting, but the steady jobs data and a 2.5 percent year-over-year increase in wages supports more interest rate hikes this year.


**Silhouette Value:** 0.5788518786430359

#### The New York Times: Three Big Questions on the Job Market, and How January’s Numbers Answer Them 
Jobs Report Get used to jobs numbers like those newly reported for the United States Friday morning — if we’re lucky.
It’s hardly enough to solve the problem of America’s missing work force, but for one month at least the numbers were pointing in the right direction.
It’s not that the new data blew the lid off expectations or pointed to some radical acceleration in job growth in the opening weeks of 2016.
How close are we to full employment?
What exactly constitutes full employment is open for debate, but it’s important to note that we are now within sight of a jobless rate that matches the low point of the 2002-2007 expansion (4.4 percent, achieved in several months in 2006 and 2007).


**Silhouette Value:** 0.6238390803337097

#### Vox: The unemployment rate is below 5 percent for the first time since 2008 
But while the unemployment rate is relatively low, slow wage growth and modest job creation seem like signs that the economy is in no danger of overheating, suggesting that the Fed might want to hold off on further interest rate increases.
Another sign of this is wage growth.
Over the past year, the average worker's hourly earnings have risen by 2.5 percent — just slightly faster than the (currently quite low) rate of inflation.
That's a bit of a slowdown — in 2015, the economy created approximately 225,000 new jobs, on average, per month.
In a really strong economic boom — the kind we enjoyed in the 1990s, for example — we should be seeing workers' wages growing much faster than the inflation rate, producing real increases in people's standards of living.


**Silhouette Value:** 0.39930111169815063


----
### 6. To avoid Zika virus, CDC says pregnant women should use condoms or abstain from sex (6 articles)

#### CNN: CDC issues new safe-sex guidelines around Zika virus 
Officials also said that pregnant women who have been exposed to Zika should talk with their doctors about testing for the virus.
Seventeen of those cases have a confirmed link to the Zika virus.
"If you're pregnant, and you're thinking about traveling to a place were Zika is spreading, please don't.
While saying that the situation with Zika is "evolving rapidly" and that much had been learned in just the past two weeks, CDC Director Dr. Tom Frieden stressed in a news briefing that the primary concern with Zika at this time is protecting pregnant women and their unborn babies from a neurological disorder known as microcephaly.
"If you live in an area where Zika is spreading and you're pregnant, please protect yourself against mosquitoes.


**Silhouette Value:** 0.1981562227010727

#### Gizmodo: New Zika Guidelines Advise Pregnant Women to Avoid Sex 
Sexual transmission of Zika is possible, and because the mosquito-borne disease has been potentially linked to birth defects, it’s of particular concern during pregnancy.
The new interim guidelines, released today, offers recommendations for men and their pregnant partners:Men who reside in or have traveled to an area of active Zika virus transmission who have a pregnant partner should abstain from sexual activity or consistently and correctly use condoms during sex (i.e., vaginal intercourse, anal intercourse, or fellatio) for the duration of the pregnancy.
The U.S. Centers for Disease Control and Prevention is asking pregnant women to abstain from sex if their partner has recently visited an area where Zika is currently active.
Pregnant women should discuss their male partner’s potential exposures to mosquitoes and history of Zika-like illness with their health care provider; providers can consult CDC’s guidelines for evaluation and testing of pregnant women.As for men and their nonpregnant sex partners, the CDC says they “might consider abstaining from sexual activity or using condoms consistently and correctly during sex.”Zika, it’s important to point out, causes relatively mild symptoms, and for most people it’s nothing to worry about.
Consequently, the CDC is advising that people take special precautions after visiting an area where Zika is currently making the rounds.


**Silhouette Value:** 0.281614750623703

#### Mashable: To avoid Zika virus, CDC says pregnant women should use condoms or abstain from sex 
While the CDC's main focus is on preventing pregnant women from being infected with the virus, the new guidelines also include recommendations for non-pregnant women, and men with non-pregnant sexual partners who live in or have traveled to Zika-affected areas.
Here is the CDC's advice for this category of people: Couples in which a man resides in or has traveled to an area of active Zika virus transmission who are concerned about sexual transmission of Zika virus may consider using condoms consistently and correctly during sex or abstaining from sexual activity.
The Centers for Disease Control and Prevention (CDC) on Friday issued new interim guidance for the prevention of sexual transmission of the Zika virus, as well as new interim guidance for health care providers to use in taking care of pregnant women and women of reproductive age.
According to the CDC: "Men with a pregnant sex partner who reside in or have traveled to an area of active Zika virus transmission and their pregnant sex partners should consistently and correctly use condoms during sex (vaginal, anal, or oral) or abstain from sexual activity for the duration of the pregnancy.
It also says pregnant women should use condoms to prevent the spread of Zika, or consider abstaining from sex for the duration of their pregnancy.


**Silhouette Value:** 0.20988470315933228

#### Reuters: CDC widens Zika guidelines for pregnant women 
(Reuters) - The U.S. Centers for Disease Control and Prevention on Friday revised its guidelines for pregnant women to include a recommendation that even those without symptoms of the Zika infection be tested after returning from affected areas.
Dallas County reported on Tuesday that the first known case of Zika contracted in the United States was a person infected after having sex with someone who had returned from Venezuela.
There is no vaccine or treatment for Zika, which has caused outbreaks in at least 26 countries in the Americas.
Experts have said that since 80 percent of those infected by the virus show no symptoms it leaves many women no way of knowing early enough to make an informed choice about their unborn child.
The updated guidelines recommend pregnant women be offered testing 2 to 12 weeks after returning from affected areas, while the agency had earlier suggested tests for those already experiencing symptoms.


**Silhouette Value:** 0.14352205395698547

#### The Hill: CDC steps up warnings about sexual transmission of Zika virus  
Out of those cases, six have been pregnant women and one child has been affected.
Twenty cases have also been reported in Puerto Rico, a U.S. territory.
The Zika virus has been rapidly spreading across South America and Central America: The CDC has issued travel warnings for more than 25 countries.
While the virus causes mild symptoms in adults, it has been linked to birth defects in babies born to Zika-infected women.
Federal health officials are stepping up their warnings of the virus, which was transmitted sexually in Texas for the first time this week — the first locally transmitted case in the U.S.


**Silhouette Value:** -0.0022366761695593596

#### The New York Times: Brazil Finds Zika Virus in Human Urine and Saliva, but Risk Is Unclear 
The findings point to the frenetic efforts by researchers in Brazil and other countries to determine the risks posed by the mosquito-borne Zika virus as it spreads through more than two dozen countries in the Americas.
RIO DE JANEIRO — Brazilian scientists said Friday that they had detected the presence of the Zika virus in samples of human saliva and urine, a first for Brazil.
French scientists had already published research last year in the Journal of Clinical Virology that located the Zika virus in human saliva following an outbreak of the virus in 2013 and 2014 in French Polynesia.
Dr. Bonaldo and officials at the institute said that they had obtained the samples from two patients with Zika-like symptoms; one was found to have the virus in urine, the other had Zika in the saliva sample.
The Brazilian researchers warned that the virus might have the potential to spread through kissing or urine, but they were quick to point out that this still remains to be proved.


**Silhouette Value:** 0.22849991917610168


----
### 7. Hillary Clinton Is Again Put on the Defensive Over Perceived Ties to Wall Street (4 articles)

#### CNN: $153 million in Bill and Hillary Clinton speaking fees, documented 
The two also reported at least $7.7 million for at least 39 speeches to big banks, including Goldman Sachs and UBS, with Hillary Clinton, the Democratic 2016 front-runner, collecting at least $1.8 million for at least eight speeches to big banks.
That time Sanders voted for Bill Clinton's plan to loosen regulation on Wall Street The Clinton campaign has been noncommittal about releasing transcripts of the paid speeches and Clinton has told reporters that she will "look into" making her remarks public.
"What being part of the establishment is, is in the last quarter, having a super PAC that raised $15 million from Wall Street, that throughout one's life raised a whole lot of money from the drug companies and other special interests," Sanders said at Thursday's Democratic debate hosted by MSNBC.
Sanders: Clinton is 'funded by Wall Street' The former secretary of state testily responded to Sanders' charges.
The analysis was made at a time when Hillary Clinton has been under scrutiny for her ties to Wall Street, which has been a major focus of Vermont Sen. Bernie Sanders on the campaign trail.


**Silhouette Value:** 0.2827678918838501

#### New Republic: Hillary Clinton Wants You to Believe She’s Tough on Wall Street. She’s Not. 
MSNBC’s Rachel Maddow pointed out that the “most frequent area of concern” reporters hear from voters leaning toward Sanders is whether Clinton has been “too close to Wall Street.” Maddow asked Clinton if she has been too dismissive of voters’ concerns in that regard, including the $675,000 that Goldman Sachs paid her in speaking fees.
Lastly, this particular effort at wrapping up the Wall Street question on Clinton’s behalf has the potential to call her opposition to Citizens United into question, given her claim that money in politics shouldn’t necessarily be read as a corruption threat.
If the circumstances warrant it, I know I will certainly use it.” It was the second time in the debate she had said she would break up big banks if “they prove to be a systemic problem,” and the second time she alluded to Obama having less-than-pure progressive credentials by Sanders’s standards, having received Wall Street money himself.
Hillary's position: if you take Wall Street $$$ & are criticized for it, that's an unfair smear.
Back in November, Hillary Clinton responded to criticisms of her Wall Street ties by pointing out that, as a senator from New York who had represented the state before and during 9/11, she was obligated to represent their interests.


**Silhouette Value:** 0.2104506939649582

#### Politico: Clinton pollster: Voters don't care about Hillary's paid-speech transcripts 
The Sanders campaign has been targeting Clinton’s paid speeches to organizations, particularly to Goldman Sachs, in recent days.
Hillary Clinton’s pollster doesn’t think voters care about her paid speeches, a point that Bernie Sanders has tried to use to illustrate the former secretary of state’s ties to Wall Street.
During Thursday night’s Democratic debate, MSNBC moderator Chuck Todd asked Clinton if she would consider releasing transcripts from her paid speeches.
Benenson also said he felt like Sanders’ invoking of Obama is an attempt to “co-opt” Clinton’s message.
“I've got a real problem with people, you know, receiving -- you know, for an hour's work to earn $200,000 plus dollars, I have a real problem with that,” Sanders to MSNBC’s “Morning Joe” Wednesday.


**Silhouette Value:** 0.13133876025676727

#### The New York Times: Hillary Clinton Is Again Put on the Defensive Over Perceived Ties to Wall Street 
“I don’t know the status, but I will certainly look into it.” For Mrs. Clinton, the key question should not be whether she received generous checks from Wall Street, but whether she did their bidding in return.
“Wall Street is an entity of unbelievable economic and political power.” In an election year fueled by the anger over the growing gap between rich and poor, Mrs. Clinton, who is widely viewed as too close to the financial sector, seems an imperfect messenger for change.
In her 2008 presidential campaign, Mrs. Clinton ran to the left of Barack Obama, angering some of her Wall Street donors with a 2007 speech in which she called for eliminating the so-called carried interest loophole, rolling back the Bush-era tax cuts on the wealthy, placing limits on chief executives’ compensation and imposing tighter regulations on derivatives.
Throughout the debate, Mrs. Clinton found herself cornered by Senator Bernie Sanders and the moderators about her ties to Wall Street, the political contributions that have bolstered her campaign account, and the hundreds of thousands of dollars in paid speaking fees she received from Goldman Sachs and other financial firms in the months after she left the State Department.
She has developed sophisticated policy proposals that many economists agree would aggressively regulate the financial sector, but they have collided with the image that Sanders supporters and other political rivals have painted of her: Wall Street’s friend and defender.


**Silhouette Value:** 0.3144300878047943


----
### 8. Thousands flee as Russian-backed offensive threatens to besiege Aleppo (5 articles)

#### CNN: Battle for Aleppo: Syrian civil war may get much worse 
And one problem looms for the rebels' backers in the West and Gulf: one of the main forces moving in to try and fight for rebel-held areas of Aleppo, is the al-Nusra Front.
Now, as thousands flee to the Turkish border from Russian airstrikes pounding the city and government forces moving in, the battle for Aleppo is again gaining significance.
All sides on the move It is too early to tell whether the move to take Nubul and Zahraa really has cut off rebel supplies into the city.
Since then it has been bombarded, seen al-Nusra Front (al Qaeda in Syria) sweep in, have moderates resist them as well as ISIS, and now faces what might be a prolonged siege.
It may change fast, but Syria's war refuses to see even the beginning of its end.


**Silhouette Value:** -0.0061761559918522835

#### Mashable: Tens of thousands of Syrians flee Aleppo as government offensive intensifies 
 So #Aleppo-city now surrounded by SAA from 3 sides (south,north,east) &no way to Bab alSalameh, only to Bab alHawa pic.twitter.com/PwJKbL9Qr4 — Jenan Moussa (@jenanmoussa) February 5, 2016 While Turkey has not allowed the thousands to enter the country, it has activated an emergency protocol allowing for the rapid processing of refugees, the BBC reported.
Russian airstrikes in Syria have mainly targeted opposition forces and led to increased violations of Turkish airspace.
The offensive, which began earlier this week, has cut an important rebel supply line to Turkey and led to a mass exodus of tens of thousands of people who are now piling up at the Syrian-Turkish border and trying to reach safety.
Backed by hundreds of Russian airstrikes, the forces of Syrian President Bashar Assad and allied militias have almost completely encircled Aleppo in what humanitarian groups warn may lead to a major humanitarian crisis.
The Syrian government offensive comes after the collapse of peace talks earlier this week.


**Silhouette Value:** -0.04249032959342003

#### Reuters: Thousands flee as Russian-backed offensive threatens to besiege Aleppo 
Aleppo, Syria’s commercial hub, has been divided for years between a section under government control and areas in the grip of rebels.
The last two days saw government troops and their Lebanese and Iranian allies fully encircle the countryside north of Aleppo and cut off the main supply route linking the city - Syria’s largest before the war - to Turkey.
“The international community must take urgent, concrete steps to address it.” NON-STOP RUSSIAN AIR STRIKES “The Russian (air) cover continues night and day, there were more than 250 air strikes on this area in one day,” Hassan Haj Ali, head of Liwa Suqour al-Jabal, a rebel group fighting in northwest Syria, told Reuters.
“Why did the opposition that left Geneva complain about the offensive in Aleppo, which is actually targeted against Jabhat al-Nusra (Nusra Front) and other radical extremist groups?” said Russia’s ambassador to the United Nations in Geneva, Alexey Borodavkin.
But swathes of the country are still in the hands of armed rebels, including Islamic State in the east, Kurdish militia in the north, and a mosaic of groups in the west who have been the target of many of the Russian air strikes.


**Silhouette Value:** 0.16239261627197266

#### The New York Times: Syrian Forces Press Aleppo, Sending Thousands Fleeing 
The bakers couldn’t find salt for their bakeries.” The blockade on the road will most likely force cargo to be rerouted through Idlib Province, where the territory around the border crossing with Turkey is controlled by the Qaeda-linked Nusra Front.
“This would provide the government with substantial leverage to eventually negotiate over the future of the city from a position of strength,” Mr. Strack wrote, adding that residents of Aleppo “would probably face a humanitarian crisis, as already seen in other besieged towns such as Madaya, but on a much larger scale.”
If the government and its allies advance farther south, they could surround rebels in Aleppo and employ the type of “starve or surrender” siege the government has used elsewhere.
Where should we go?” A man who said he was from Homs Province, several hundred miles to the south, said he had fled to one town after another in Aleppo Province, always chased by bombardments and shelling.
Airstrikes and bombardments in the area may indicate that government forces plan on pressing south to cut the Idlib road as well and surround Aleppo.


**Silhouette Value:** 0.018008152022957802

#### Vice News: Russia Says It's 'in Bad Taste' to Blame It for Failed Syria Peace Talks 
Related: The Syrian Regime Is Close to a Victory That Could Turn the War The Russian-aided offensive in Aleppo was condemned by opposition representatives gathered in Geneva for what were supposed to be UN-brokered negotiations.
"On the one hand, the Syrian regime claims to discuss peace in Geneva, and on the other hand, it intensifies its military offensive against opposition groups with which it is supposed to discuss, and imposes on the city of Aleppo an unprecedented torrent of fire," said Delattre.
While he said Russia was "upset" that the Geneva talks had let out, Churkin said Moscow was "preparing some ideas on how to move things forward" to be delivered to the International Syria Support Group (ISSG) when it meets in Munich on February 11.
"The Syrian regime with the support of Russia are responsible for the situation as it is and for the fact that the opposition simply can simply not negotiate with… a gun to their head."
Related: The Syrian Regime Is Close to a Victory That Could Turn the War France's UN ambassador Francois Delattre also spoke after the council session on Friday, and blasted the government of Bashar al-Assad and Russia.


**Silhouette Value:** 0.31655505299568176


----
### 9. Friday's spectacular volcanic eruption in Japan, in one GIF (5 articles)

#### Gizmodo: Sakurajima Volcano Erupts in a Spectacular Display of Lightning and Lava 
Residential areas at the foot of the volcano “could be gravely affected,” as noted in the Japan Times, but Kazuhiro Ishihara from Kyoto University told NHK that people living nearby needn’t worry because video images show rocks flying “only” a mile or so from the mountain’s top.
Mount Sakurajima is an active volcano, and ongoing eruptions are expected.
Earlier today, a volcano near the city of Kagoshima in Japan erupted, spewing lava and hurtling rocks nearly two miles away.
No reports of injuries have been reported, but officials with Japan’s Meteorological Agency have cordoned off the area, expanding an existing no-entry zone around the volcano to a 1.2 mile (2 km) radius.
The volcano is just 30 miles from the Sendai nuclear plant, but officials says there’s no immediate cause for concern.


**Silhouette Value:** 0.7646558284759521

#### Mashable: Famous Japan volcano erupts spectacularly with fiery lightning 
 A well-known volcano erupted Friday evening in southern Japan with a fiery blast that sent lava rolling down its slope.
Japan's Meteorological Agency said Sakurajima on the island of Kyushu erupted at around 7 p.m. (1000 GMT).
Japanese television showed an orange burst out of the side of the volcano, near the summit, accompanied by lightning-like flashes.
The 2014 eruption of Mount Ontake in central Japan killed 57 people.
Friday's eruption, while dramatic, was average compared to Sakurajima's past eruptions, Kyoto University volcanologist Kazuhiro Ishihara told NHK.


**Silhouette Value:** 0.7341523170471191

#### Refinery 29: Mount Sakurajima Japan Volcano Erupt Lava 
While the scene, broadcast on the local news, was certainly dramatic, one volcanologist told a public broadcaster that Friday's eruption wasn't anything special compared to what they've seen before from Mount Sakurajima, the AP reported.
A volcano in southern Japan erupted in a fashion that seemed made for the movies on Friday, creating a burst of lava, lightning, and smoke that was captured on video.The blast, which Japan's Meteorological Agency said occurred at about 7 p.m. local time, was expected to send rocks flying more than a mile, according to the Associated Press.
Watch the dramatic video of the eruption below:
That might not be so surprising considering that Japan is, as the AP notes, home to more than 100 volcanos.
The volcano on the island of Kyushu is located just 31 miles from a nuclear power plant, according to The Japan Times.


**Silhouette Value:** 0.7107035517692566

#### Vox: Friday's spectacular volcanic eruption in Japan, in one GIF 
Here's the eruption, slightly sped up, in GIF form: Japan's meteorological agency issued a level 3 warning, telling people to stay away from the volcano.
Volcanic eruptions aren't unusual in Japan, which has more than 100 active volcanoes.
The full video shows the scope of the dramatic eruption:
Japan's Sakurajima volcano, after months of calm, exploded at 7 pm Japan time Friday, creating a spectacular eruption against the night sky.
But since September, its last major eruption, the volcano had been unusually quiet, according to the Associated Press.


**Silhouette Value:** 0.7968440651893616

#### Wired: Sakurajima Lets Out a Spectacular Eruption After Months of Quiet 
I've seen some brief reports of a lava flow from the eruption as well, but this would be a bit of a surprise based on Sakurajima's usual eruption style.
If you watch the – video of the eruption, you can see incandescent volcanic bombs and ash being hurled hundreds of meters into the air—and the eruption even generated some of its own lightning.
When it is in an eruptive period, the volcano can regularly throw volcanic bombs and ash multiple kilometers from the active vent—this eruption was doing just that.
Things had been quiet at Japan's Sakurajima for the past few months, but today the volcano unleashed an impressive explosive eruption.
Images and video of the eruption show those glowing blocks of debris moving downhill, but those are not lava flows.


**Silhouette Value:** 0.7400749921798706


----
### 10. Sick Baby Sea Lion Found in California Restaurant's Booth (5 articles)

#### CNN: Sea lion pup rescued after napping at restaurant 
 (CNN)An underfed but overly cute sea lion cub is getting a lot of TLC after it waddled into an empty upscale California restaurant without a reservation.
SeaWorld said the cutie likely sought food and shelter from a higher than normal tide.
A SeaWorld team was called to the restaurant to pick up the malnourished sea lion.
When food is farther away, the mothers are away from the pup too long in search of food, and return with little food or too few nutrients for a growing sea lion.
The federal government last year reported a record number of sea lion "strandings."


**Silhouette Value:** 0.7988306879997253

#### Mashable: Baby sea lion parks itself at upscale restaurant for a little brunch 
A hungry sea lion pup, weighing only 20 pounds, sat itself down at The Marine Room, an upscale seafood restaurant in San Diego, California, Thursday morning.
The cute baby sea lion certainly delighted the guests who were dining around it.
After taking a few more pictures, the restaurant contacted a Sea World San Diego team to come and rescue the pup.
Many sea lion pups have been greatly effected by changes in their environment and El Niño.
The National Oceanic and Atmospheric Association has observed that the unusually warm water forces mother sea lions to hunt further out for prey — leaving their pups (who are dependent on their milk) for longer periods of time.


**Silhouette Value:** 0.7580787539482117

#### People: Sick Baby Sea Lion Found in California Restaurant's Booth 
The team found the 8-month-old female sea lion alert but suffering from several health problems, including dehydration and malnourishment.
The employees of Marine Room came to work on Thursday morning to find a sea lion pup lying in one of the ocean view booths, reports The Los Angeles Times.
Rescuers took the sea lion to their center, where they gave her fluids and treatment.
Concerned for the animal, who was barely moving, the restaurant’s staff called a SeaWorld rescue team for help.
A La Jolla, California, sea lion sought refuge in a restaurant without a reservation.


**Silhouette Value:** 0.8151916265487671

#### The New York Times: A Hungry Sea Lion Wanders Into a Restaurant ... 
About 8 a.m. Thursday, Leslie Tovar, a manager for the Shores Restaurant, another of the club’s restaurants, was on the grounds when she got a call from a custodian at the Marine who was “vacuuming up the floor and happened to come across a baby sea lion that matched the interior very well.” “He said there was a sea lion in the dining room,” Ms. Tovar said in a telephone interview Friday.
“He was a little bit early for his high tide breakfast reservation.” The pup was eventually rescued and taken to San Diego’s SeaWorld.
At one point, he said, the sea lion pup looked out the window toward the ocean, as if to say, “Can I go back now?” An earlier version of this article misidentified one of the California counties where experts have noticed a rise in the number of stranded sea lions.
A hungry sea lion pup wandered off the beach and into a fancy seaside San Diego restaurant Thursday morning, took one of the best seats in the house and peered out the window at the waves as if preparing to order a big plate of sardines.
The restaurant, the Marine Room, does not open for dinner until 5:30 p.m. — unless it is offering one of its special “high-tide breakfasts.” Bernard Guillas, the executive chef at the restaurant, posted photos of the pup, curled up or looking out the window, on his Facebook page Thursday.


**Silhouette Value:** 0.7855299115180969

#### Vice: A Starving Sea Lion Dragged Itself to a San Diego Restaurant Booth 
It is now being cared for at SeaWorld's Animal Rescue Center in San Diego.
The restaurant's owners promptly called SeaWorld for help, and workers came to the aid of the pup.
The sea lion is the 47th the company says it has rescued this year, along with other marine mammals.
Marine biologists have collected thousands of sick and dead pups as a result.
It wandered in searching for food and parked itself in a booth of the Marine Room, an upscale restaurant, the AP reports, creating an image that was simultaneously adorable and also pretty grim, since the pup was severely underweight and dehydrated.


**Silhouette Value:** 0.7447945475578308



## Summaries (by the best clusters):

----
### 1. Manuel Valls warns terrorist threat is here to last (2 articles)

#### Business Insider: Manuel Valls warns terrorist threat is here to last
Valls, defending state of emergency rules that have allowed police conduct thousands of house searches in just a few months, also said over 2,000 French residents were believed to be involved with jihadi networks based in Syria and Iraq.
By Elizabeth Pineau PARIS (Reuters) - Thousands of house searches since November's Islamist attacks in Paris have helped foil another terrorist plot, French Prime Minister Manuel Valls said on Friday as his government sought to extend emergency rule.
In 2015, 15 terror plots were foiled by the French security services, he said.
In the three months since the attacks on Paris, police have carried out 3,289 house searches, placed 341 people in custody, put 407 under house arrest and confiscated 560 weapons, 42 of them war-grade, the prime minister said.
"The terrorist threat is here, and here to last," Valls told the National Assembly, where the government is asking lawmakers to extend the state of emergency to the end of May and amend the Constitution so people convicted on terror charges can be stripped of their citizenship.


**Silhouette Value:** 0.972147524356842

#### Reuters: French PM defends emergency rule, says terror threat 'here to last'
Valls, defending state of emergency rules that have allowed police conduct thousands of house searches in just a few months, also said over 2,000 French residents were believed to be involved with jihadi networks based in Syria and Iraq.
PARIS (Reuters) - Thousands of house searches since November’s Islamist attacks in Paris have helped foil another terrorist plot, French Prime Minister Manuel Valls said on Friday as his government sought to extend emergency rule.
In 2015, 15 terror plots were foiled by the French security services, he said.
In the three months since the attacks on Paris, police have carried out 3,289 house searches, placed 341 people in custody, put 407 under house arrest and confiscated 560 weapons, 42 of them war-grade, the prime minister said.
“The terrorist threat is here, and here to last,” Valls told the National Assembly, where the government is asking lawmakers to extend the state of emergency to the end of May and amend the Constitution so people convicted on terror charges can be stripped of their citizenship.


**Silhouette Value:** 0.9721491932868958


----
### 2. Whooping Cough Booster Shot May Offer Only Short-Term Protection (2 articles)

#### CNN: Whooping cough protection fades fast after Tdap, study finds
Rethinking vaccine strategies The researchers found that the pertussis cases in the study, whether in vaccinated or unvaccinated children, were mild or moderate.
One possibility could be to give children Tdap only if a pertussis outbreak is on the horizon, which would require local health departments and medical centers to do regular surveillance, she added.
By now, it is not that surprising that protection against pertussis from Tdap is short lived, said Dr. James D. Cherry, distinguished research professor in the David Geffen School of Medicine at the University of California, Los Angeles.
She and her colleagues conducted a study showing the risk of pertussis increased by additional 42% for each year after DTaP immunization.
The bigger concern than improving immunity among adolescents, who generally have mild disease from pertussis, is how to protect babies -- especially those younger than 3 months old -- in whom pertussis can be deadly, Cherry said.


**Silhouette Value:** 0.8583424687385559

#### The New York Times: Whooping Cough Booster Shot May Offer Only Short-Term Protection
The rapidly fading effectiveness of the pertussis booster vaccine may help explain recent widespread outbreaks of whooping cough.
The United States stopped using a whole-cell pertussis vaccine in the 1990s and began using an acellular version called DTaP.
Five vaccinations are given during childhood, and a booster vaccine, called the Tdap, is given to adolescents and adults.
Or we could vaccinate whenever there is an outbreak.” Dr. Klein added that the vaccination of pregnant women is effective in preventing pertussis in newborns, and that all pregnant women should get the Tdap vaccine in the third trimester of pregnancy.
Researchers looked at 1,207 pertussis cases among children who had had the acellular vaccine in childhood.


**Silhouette Value:** 0.8615314364433289


----
### 3. Morocco turns on what will become the world’s largest solar power plant (2 articles)

#### Gizmodo: Morocco Switches on First Phase of the World's Largest Solar Plant
With the extra mirrors in place, the plant will generate 580 megawatts of electricity—enough to provide energy for 1.1 million people.But, as our own George Dvorsky has pointed out, that wasn’t always to be the case.
The new installation already creates 160 megawatts of power and is expected to grow to cover 6,000 acres by 2018—making it the largest in the world.The first wave of power production is known as Noor 1.
Yesterday, Morocco switched on the first section of its new Ouarzazate solar power plant.
When those sections come online, the whole plant will cover an areas of over 6,000 acres, which is larger than the country’s capital city of Rabat.
Interventions by the African Development Bank and the Moroccan government saved the project, though, and are now using it to meet Morocco’s own power demands.


**Silhouette Value:** 0.8335263729095459

#### The Verge: Morocco turns on what will become the world’s largest solar power plant
It's already one of the biggest solar power grids in the world, capable of being seen from space.
Morocco gets about 3,000 hours of sunlight per year, so there will be plenty of solar energy to harness.
The plant already spans thousands of acres and is capable of generating up to 160 megawatts of power.
It's just the first phase of a planned project to bring renewable energy to millions Right now, the solar farm is made up of 500,000 curved mirrors, each standing at about 40 feet tall.
It's all part of Morocco's plan to get up to 42 percent of its power from renewable energies at home, such as solar, wind, and hydropower.


**Silhouette Value:** 0.8566669225692749


----
### 4. The Pentagon Just Released 200 Long-Awaited Photos Related to Detainee Abuse (4 articles)

#### CNN: Pentagon releases 198 photos of detainees
RELATED: Judge orders Pentagon to release pictures of detainees  The release of the photos was the result of a Freedom of Information Act lawsuit the American Civil Liberties Union brought against the Defense Department in 2004.
The ACLU also argued that the Defense Department must individually review each photo and explain why its release would put Americans in danger to attempt to justify holding back the information.
Washington  (CNN)The Pentagon released 198 photos Friday, several of which appear to show injuries suffered by detainees after allegedly experiencing abuse while held in Iraq and Afghanistan during the George W. Bush administration.
"We're still pursuing our case for the remaining secret photos, which we think are very important because a complete picture is necessary to show the full extent of the abuse," ACLU spokesman Josh Bell told CNN.
"The disclosure of these photos is long overdue, but more important than the disclosure is the fact that hundreds of photographs are still being withheld," said ACLU Deputy Legal Director Jameel Jaffer.


**Silhouette Value:** 0.8378629684448242

#### The Hill: Pentagon releases 198 photos of alleged detainee abuse 
“The disclosure of these photos is long overdue, but more important than the disclosure is the fact that hundreds of photographs are still being withheld,” ACLU deputy legal director Jameel Jaffer said in a statement.
The photos were released as a result of a lawsuit from the American Civil Liberties Union, which is pressing the Pentagon to release an entire trove of about 2,000 photos.
The Pentagon on Friday released nearly 200 photos showing bruises, cuts and scars suffered by detainees who were alleged to have suffered abuse.
In 2009, the Obama administration promised to release the photos, but Congress passed a law that allowed them to remain classified if the Defense secretary certified their release would jeopardize national security.
The 198 photos released Friday were part of investigations that found 14 substantiated allegations of abuse and 42 unsubstantiated ones, according to a Pentagon statement.


**Silhouette Value:** 0.7947308421134949

#### The New York Times: Pentagon Releases Small Portion of Photos From Detainee Abuse Cases
That case has resulted in the release of many documents and memos about abusive interrogation practices, but it ran into turbulence in 2009 over the photos.
After making public Bush-era memos about torture in response to the same lawsuit, the Obama administration was initially going to comply with an order to release the first group of pictures from the detainee abuse investigations as well.
What’s in the 1,800 photographs the government still hasn’t released?” Cmdr.
The photographs are a focus of a Freedom of Information Act lawsuit filed in 2004 by the American Civil Liberties Union in the wake of the Abu Ghraib prison torture scandal.
However, the military is continuing to block the disclosure of about 1,800 other photos from the same criminal investigations, saying that their release would endanger American service members serving abroad.


**Silhouette Value:** 0.8243087530136108

#### Vice News: The Pentagon Just Released 200 Long-Awaited Photos Related to Detainee Abuse
The photographs, taken by military investigators in 2004 and 2006, were released in response to a long-running Freedom of Information Act (FOIA) lawsuit filed against the US government by the ACLU, which has sought records pertaining to the treatment of detainees in custody of the US military.
As VICE News previously reported, a review we conducted of Army criminal investigative reports that probed the detainee abuse allegations contained detailed descriptions of the pictures that are still being withheld.
Related: Up to 2,100 Photos of US Soldiers Abusing Prisoners May Soon Be Released "These photos fail to show a single act of abuse which the government's own records describe as having taken place," Dr. Vincent Iacopino, medical director of Physicians for Human Rights (PHR), a party to the lawsuit seeking the release of the images, said in a statement.
The Department of Defense (DOD) released 198 photographs on Friday from Army and Navy criminal investigative files related to allegations of detainee abuse in Iraq and Afghanistan, the first time the US government has ever disclosed such images.
Related: A Judge Has Ordered the US Government to Release Thousands of Detainee Abuse Photos More than a dozen other photographs, Army criminal investigative reports say, show US Army soldiers in Afghanistan pointing assault rifles and pistols at the heads and backs of hooded and bound detainees, and soldiers kicking and punching detainees.


**Silhouette Value:** 0.8539382815361023


----
### 5. Edgar Mitchell, Apollo 14 astronaut, moon walker, dies (2 articles)

#### CNN: Edgar Mitchell, Apollo 14 astronaut, moon walker, dies
Mitchell was part of the support crew for the Apollo 9 and Apollo 10 missions.
"On behalf of the entire NASA family, I would like to express my condolences to the family and friends of NASA astronaut Edgar Mitchell," NASA Administrator Charles Bolden said in a statement.
Mitchell, who was the lunar module pilot, found the trip to be a profound experience.
Mitchell became the sixth man to walk on the lunar surface.
"He believed in exploration, having been drawn to NASA by President Kennedy's call to send humans to the moon.


**Silhouette Value:** 0.8249703645706177

#### The New York Times: Edgar D. Mitchell, Sixth Moonwalking Astronaut, Dies at 85
Commander Mitchell, who had a doctorate in flight sciences, had long maintained a parallel interest in the study of consciousness, a fascination that was heightened during his return from the moon.
Apollo 14 was Commander Mitchell’s only spaceflight.
For the 14th mission, scientists were counting on Commander Mitchell and Captain Shepard to be the first astronauts to return with a large collection of rocks from high elevations, where some might be old enough to provide clues to the moon’s origin and its evolution.
NASA announced his death, which came one day before the 45th anniversary of his moon landing.
The Apollo 14 flight, launched on Jan. 31, 1971, took Commander Mitchell and his fellow Navy officer, Capt.


**Silhouette Value:** 0.8188241720199585


----
### 6. California doctor gets 30 years to life in landmark overdose case (2 articles)

#### CNN: Southern California doctor sentenced in overdose deaths
The doctor was charged with prescribing addictive narcotics to patients who didn't need them.
Medical experts and law enforcement officials blame reckless and criminal prescribing of opiate-based painkillers and other potent narcotics by doctors like Tseng for contributing to the problem.
Tseng is the only doctor in the United States to be convicted of murder for recklessly prescribing drugs, said Los Angeles County Deputy District Attorney John Niedermann, the lead prosecutor on the case.
Tseng's case is in some ways emblematic of the role so-called dirty doctors play in creating and sustaining addicts, according to law enforcement officials interviewed by CNN.
Assistant U.S. Attorney Ann Luotto Wolf, who prosecuted both doctors, said she focused on drug dealing charges because they carried sufficiently stiff sentences and were not subject to cause-of-death determinations, which can be very complicated in doctor prosecutions.


**Silhouette Value:** 0.7968884706497192

#### Reuters: California doctor gets 30 years to life in landmark overdose case
At trial, prosecutors pointed to nine overdose deaths associated with Tseng’s practice in less than three years, during which they said she had made $5 million from her clinic, dispensing addictive medications to patients unnecessarily.
Prosecutors said Tseng’s conviction after a six-week trial marked the first time in which a U.S. physician was found guilty of murder for over-prescribing drugs.
Defense lawyers argued at trial that her patients put themselves in jeopardy by taking drug dosages far in excess of what Tseng prescribed.
The case against Dr. Hsiu Ying “Lisa” Tseng, 46, comes amid what public health officials describe as a national epidemic of drug abuse.
A Los Angeles jury in October convicted Tseng of three counts of second-degree murder in a case prosecutors said showed she put greed above patients’ wellbeing.


**Silhouette Value:** 0.8248116970062256


----
### 7. Google AI Will Compete Against Go World Champion Lee Sedol On YouTube (2 articles)

#### Gizmodo: Google AI Will Compete Against Go World Champion Lee Sedol On YouTube
DeepMind founder Demis Hassabis announced the competition in a tweet.The match between Sedol and AlphaGo will have many similarities to the famous 1996 chess match between chess grandmaster Gary Kasparov and IBM’s Deep Blue computer.
It’s official: The world champion of Go, Lee Sedol, will face off against Google Deepmind’s powerful artificial intelligence, called AlphaGo.A week ago, Google DeepMind’s team claimed to have built the best AI for the game in a scientific paper, and issued a public challenge to Sedol.
In that match, IBM’s Deep Blue artificial intelligence was the first to defeat a professional chess grandmaster.In the upcoming match between Sedol and AlphaGo, DeepMind’s artificial intelligence will reason through many more combinations of moves than IBM’s computer ever did.
DeepMind hopes it can prove that a powerful artificial intelligence is capable of beating the best human player in one of the most mathematically complicated board games ever created.The match between Sedol and DeepMind will be broadcast live on YouTube beginning March 9.
By comparison, chess only has 10 to the power of 60 possible scenarios.AlphaGo already successfully defeated three-time European Go champion Fan Hui in a series of five matches, but Sedol is considered to be a different challenge entirely.


**Silhouette Value:** 0.8010242581367493

#### The Verge: Google’s AI will battle Go world champion live on YouTube
However, the computer only beat the game's European champion, and in March, DeepMind's AlphaGo AI will take on the world champion, with YouTube live streaming the series of games.
DeepMind's founder Demis Hassabis announced details about the competition on Twitter: the five-game matchup will take place in Seoul on March 9th to the 15th (there'll be a game a day apart from on the 11th and 14th), with AlphaGo facing off against the world's top-ranked player, South Korea's Lee Sedol, for a $1 million prize.
"I have heard that Google DeepMind’s AI is surprisingly strong and getting stronger, but I am confident that I can win at least this time," he said in a press statement.
Earlier this month, Google's AI company DeepMind announced that it had built a system of neural networks capable of beating a champion Go player.
DeepMind's win against Hui has been described as a "decade earlier than expected," and any setback would likely be temporary — just as it was with Deep Blue.


**Silhouette Value:** 0.7942807674407959


----
### 8. Outrage Grows as Italy Investigates Student’s Death in Egypt (2 articles)

#### Reuters: Italian student killed in Egypt criticized Cairo govt. in articles
“But we ask for the truth.” With Italian police specialists en route to Cairo to participate in the investigation, a spokesman at Egypt’s Foreign Ministry said on Facebook both countries had “a mutual desire to uncover the reasons behind this incident”.
ROME (Reuters) - An Italian student found dead by a roadside in Cairo with cigarette burns and other signs of torture on his body had written articles critical of the Egyptian government, according to the Italian newspaper that published them.
On Thursday, the Italian Foreign Ministry “urgently” summoned the Egyptian Ambassador to express concern about the student’s death, and Prime Minister Matteo Renzi telephoned Sisi, asking for a joint investigation and the swift return of his body to Italy.
“President Abdel Fattah al-Sisi presides over the Egyptian parliament with the highest number of police and military personnel in the history of the country, and Egypt ranks among the worst offenders with respect to press freedom,” Regeni wrote in the first paragraph of the story.
Regeni, a postgraduate student at Britain’s University of Cambridge, disappeared on the fifth anniversary of the start of the uprising that ended former leader Hosni Mubarak’s 30-year rule.


**Silhouette Value:** 0.8160098195075989

#### The New York Times: Outrage Grows as Italy Investigates Student’s Death in Egypt
There was no indication that Mr. Regeni’s writing led to his death, but the article contributed to the broader Italian outrage over Mr. Regeni’s injuries as news outlets pointed an accusatory finger at the Egyptian security forces.
But in Mr. Regeni’s case, the strongest suspicions have focused on the security forces because the circumstances of his disappearance match those of many Egyptians who have vanished into secretive detention facilities, run by the security forces, where torture is common.
That was proof, said Egypt’s ambassador to Rome, Amr Helmy, of the two countries’ “mutual desire to identify the culprits who committed this criminal, shameful act.” But before the Italians had landed, Egyptian investigators claimed to have made a breakthrough that, to critics, raised the specter of a cover-up.
CAIRO — The furor surrounding the death of an Italian student whose body was discovered Wednesday on an Egyptian roadside grew Friday as Italian investigators flew to Cairo to help find his killers, and it emerged that the young man had secretly written from Egypt for a left-wing Italian newspaper.
The furor over Mr. Regeni received limited coverage on Egypt’s pro-government television channels, although some lashed out at foreign criticism, which was painted as part of an Islamist plot to destroy Egypt’s relations with Italy.


**Silhouette Value:** 0.7784402966499329


----
### 9. US traffic fatalities rose slightly last year, but don't panic just yet (2 articles)

#### Reuters: U.S. traffic deaths increase sharply in 2015
“It’s time to drive behavioral changes in traffic safety and that means taking on new initiatives and addressing persistent issues like drunk driving and failure to wear seat belts.” The fatality rate was 1.1 deaths per 100 million miles traveled in 2015, compared with 1.05 deaths per 100 million miles the previous year.
The U.S. National Highway Traffic Safety Administration, a branch of the Transportation Department, did not immediately have an explanation for the increase in traffic deaths to an estimated 26,000 in 2015, which is the highest level since 2008.
WASHINGTON, Feb 5 (Reuters) - Traffic deaths in the United States rose 9.3 percent in the first nine months of 2015 compared to same period a year before as low gasoline prices increased road travel, the U.S. Transportation Department said on Friday.
The pace of road deaths is greater than the growth in vehicle travel.
U.S. traffic deaths had been falling with a 1.2 percent decline in 2014 and more than 22 percent from 2000 to 2014.


**Silhouette Value:** 0.7950366735458374

#### The Verge: US traffic fatalities rose slightly last year, but don't panic just yet
If people are driving more miles, it's natural that traffic accidents (and then fatalities) would increase as well, as long as technology and behavior remain the same.
In 2005, we saw traffic fatalities reach 1.43 per 100 million vehicle miles, well above what we're seeing these days.
By any measure, cars — and the people who ride in them — are safer than they've ever been, even with this slight uptick in fatalities.
And, unsurprisingly, traffic deaths per vehicle mile rose as well, from 1.05 per 100 million vehicle miles for the first nine months of 2014 up to 1.10 for the first nine months of 2015, according to data from the National Highway Traffic Safety Administration (NHTSA).
Traffic fatalities in the US, for the first nine months of 2015, rose 9.3 percent from the year prior.


**Silhouette Value:** 0.7798502445220947


----
### 10. This secret code will let you play chess on Facebook Messenger (2 articles)

#### Mashable: This secret code will let you play chess on Facebook Messenger
From the amazingly intricate easter egg department: Facebook has a hidden, fully functional, graphical game of chess embedded into its Messenger, and anyone can play.
To move a pawn from the starting position, for example, you'd type "@fbchess Pb4".
Two games of Facebook's hidden chess started in two Messenger windows.
To start a game, open a Messenger conversation with someone and type "@fbchess play".
There's quite a bit more to it; to get the full list of commands, type "@fbchess help" into Messenger.


**Silhouette Value:** 0.7824823260307312

#### The Verge: Here's how to play Facebook Messenger's secret chess game
So, "@fbchess Pe4" would move your pawn (P) to square E4, for example.
Want to play chess with your friends on Facebook?
Type "@fbchess help" You can also type "@fbchess help" for instructions, but here are the basics: K is for king, Q for queen, B for bishop, N for knight, R for rook, and P for pawn.
B2xc5 will have your bishop in the second rank, or horizontal row, to square c5); the command "=" to promote a pawn to a queen (e.g.
You can also use "@fbchess resign" to end a match, show you and your friend's vital statistics with "@fbchess stats," and pick which color you start with by typing "@fbchess play white" or "@fbchess play black."


**Silhouette Value:** 0.7903338074684143

